In [1]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import torch
#from transformers import MarianMTModel, MarianTokenizer
import pandas as pd
# from googletrans import Translator
#from groq import Groq
from IPython.display import display, HTML
import sacrebleu
from google import genai  # Gemini
from google.genai import types
from nltk.translate.bleu_score import sentence_bleu


# ---------------- Gemini Client ----------------
gemini_client = genai.Client(api_key="GEMINI_API_KEY")

# ---------------- Groq Client ----------------
# client = Groq(api_key="GROQ_API_KEY")

In [8]:
groq_models = [
    "meta-llama/llama-guard-4-12b",
    "meta-llama/llama-4-maverick-17b-128e-instruct",
    "deepseek-r1-distill-llama-70b"
]

def groq_translate(model_name, prompt, text):
    full_prompt = f"{prompt}\n\n{text.strip()}"
    completion = client.chat.completions.create(
        model=model_name,
        messages=[{"role": "user", "content": full_prompt}]
    )
    return completion.choices[0].message.content
    
def translate_en_to_mr(text: str) -> str:
    inputs = tokenizer([text], return_tensors="pt", padding=True, truncation=True).to(device)
    translated_tokens = model.generate(**inputs, max_length=128)
    translated_text = tokenizer.decode(translated_tokens[0], skip_special_tokens=True)
    return translated_text

def gemini_translate(text: str) -> str:
    response = gemini_client.models.generate_content(
        model="gemini-2.0-flash",
        contents=f"Translate the following English legal text into Marathi:\n\n{text}"
    )
    return response.text

def gemini_translate_pro(text: str) -> str:
    response = gemini_client.models.generate_content(
        model="gemini-2.5-pro",   # 🔄 switched to 2.5 Pro
        contents=f"Translate the following English legal text into Marathi:\n\n{text}"
    )
    return response.text

dataset = [
    {
        "english_text" : """
SALE DEED
This Sale Deed is executed on this ___ day of ____, 20 at [Place], Maharashtra.
BY AND BETWEEN
1. [Seller Name], aged about ___ years, residing at [Address], hereinafter referred to as the "SELLER" (which expression shall, unless repugnant to the context or meaning thereof, include his/her heirs, legal representatives, executors, administrators, and assigns).
AND
2. [Buyer Name], aged about ___ years, residing at [Address], hereinafter referred to as the "BUYER" (which expression shall, unless repugnant to the context or meaning thereof, include his/her heirs, legal representatives, executors, administrators, and assigns).
The SELLER and the BUYER are collectively referred to as the “Parties”.
________________________________________
WHEREAS:
1.	The SELLER is the absolute and lawful owner of the plot of land bearing Survey No. [____], admeasuring [____] sq. m./sq. ft., situated at [Village/Town, Taluka, District, Maharashtra], hereinafter referred to as the "Said Property".
2.	The SELLER has agreed to sell the Said Property to the BUYER for a total consideration of ₹[Amount] (Rupees [Amount in words]).
3.	The BUYER has agreed to purchase the Said Property from the SELLER for the said consideration, free from all encumbrances, liens, charges, mortgages, or litigation.
________________________________________
NOW THIS DEED WITNESSETH AS FOLLOWS:
1.	Sale and Transfer:
The SELLER, for the consideration mentioned above, hereby sells, transfers, and conveys the Said Property to the BUYER absolutely, together with all rights, easements, and appurtenances thereto.
2.	Consideration:
The total sale consideration of ₹[Amount] has been received by the SELLER from the BUYER by way of [mode of payment: cheque/cash/bank transfer], the receipt of which the SELLER hereby acknowledges.
3.	Possession:
The SELLER hereby delivers physical and lawful possession of the Said Property to the BUYER on the execution of this Deed.
4.	Title and Warranty:
The SELLER assures that the Said Property is free from all encumbrances, legal disputes, claims, or mortgages. The SELLER further agrees to indemnify the BUYER against any loss arising out of any defect in the title of the Said Property.
5.	Expenses and Registration:
All expenses relating to stamp duty, registration charges, and other incidental costs shall be borne by the [Buyer/Seller mutually agreed].
6.	Governing Law:
This Sale Deed shall be governed by and construed in accordance with the laws of India, and specifically, the Maharashtra Stamp Act and Transfer of Property Act, 1882.
________________________________________
IN WITNESS WHEREOF, the SELLER and the BUYER have executed this Sale Deed on the day, month, and year first above written.
Signed, Sealed, and Delivered by:
SELLER:
(Signature) ___________________
Name: _______________________
Address: ____________________
BUYER:
(Signature) ___________________
Name: _______________________
Address: ____________________
WITNESSES:
1.	Name: ___________________ Signature: _______________ Address: _______________
2.	Name: ___________________ Signature: _______________ Address: _______________

""",
        "reference_translation" : """विक्री करार
हे विक्री करार २० तारखेला महाराष्ट्रातील [स्थान] येथे ____ दिवशी अंमलात आणले जात आहे .
द्वारे आणि दरम्यान
१. [विक्रेत्याचे नाव] , वय सुमारे ___ वर्षे, [पत्ता] येथे राहणारा , यापुढे "विक्रेता" म्हणून संदर्भित (या अभिव्यक्तीमध्ये, संदर्भ किंवा अर्थाच्या विरुद्ध नसल्यास, त्याचे/तिचे वारस, कायदेशीर प्रतिनिधी, कार्यकारी, प्रशासक आणि नियुक्त केलेले यांचा समावेश असेल).
आणि
२. [खरेदीदाराचे नाव] , वय सुमारे ___ वर्षे, [पत्ता] येथे राहणारा , यापुढे "खरेदीदार" म्हणून संदर्भित (या अभिव्यक्तीमध्ये, संदर्भ किंवा अर्थाच्या विरुद्ध नसल्यास, त्याचे/तिचे वारस, कायदेशीर प्रतिनिधी, कार्यकारी, प्रशासक आणि नियुक्त केलेले यांचा समावेश असेल).
विक्रेता आणि खरेदीदार यांना एकत्रितपणे "पक्ष" असे संबोधले जाते.
________________________________________
जिथे:
1.	[गाव/शहर, तालुका, जिल्हा, महाराष्ट्र] येथे असलेल्या , यापुढे "उक्त मालमत्ता" म्हणून उल्लेख केलेल्या, सर्व्हे क्रमांक [____] असलेल्या [____] चौरस मीटर/चौरस फूट आकाराच्या भूखंडाचा पूर्ण आणि कायदेशीर मालक आहे .
2.	₹[रक्कम] (रुपये [रक्कम शब्दात]) मध्ये विकण्यास सहमती दर्शविली आहे .
3.	खरेदीदाराने विक्रेत्याकडून सदर मालमत्ता सर्व भार, धारणाधिकार, शुल्क, गहाणखत किंवा खटल्यापासून मुक्त करून, सदर मोबदल्यात खरेदी करण्यास सहमती दर्शविली आहे.
________________________________________
आता हे कृत्य खालीलप्रमाणे साक्ष देते:
1.	विक्री आणि हस्तांतरण: 
वर उल्लेख केलेल्या मोबदल्यात, विक्रेता याद्वारे सर्व हक्क, सुविधा आणि उपयोजनांसह, सांगितलेली मालमत्ता पूर्णपणे खरेदीदाराला विकतो, हस्तांतरित करतो आणि हस्तांतरित करतो.
2.	विचार: 
विक्रेत्याला खरेदीदाराकडून [पेमेंटची पद्धत: चेक/रोख/बँक हस्तांतरण] द्वारे एकूण विक्री विचारात घेतलेला ₹[रक्कम] मिळाला आहे , ज्याची पावती विक्रेता याद्वारे मान्य करतो.
3.	ताबा: 
विक्रेता याद्वारे या कराराच्या अंमलबजावणीनंतर खरेदीदाराला उक्त मालमत्तेचा भौतिक आणि कायदेशीर ताबा देतो.
4.	मालकी हक्क आणि हमी: 
विक्रेता खात्री देतो की सदर मालमत्ता सर्व भार, कायदेशीर वाद, दावे किंवा गहाणखतांपासून मुक्त आहे. विक्रेता सदर मालमत्तेच्या मालकी हक्कातील कोणत्याही दोषामुळे उद्भवणाऱ्या कोणत्याही नुकसानाविरुद्ध खरेदीदाराला भरपाई देण्यास सहमत आहे.
5.	खर्च आणि नोंदणी: 
मुद्रांक शुल्क, नोंदणी शुल्क आणि इतर आनुषंगिक खर्चाशी संबंधित सर्व खर्च [खरेदीदार/विक्रेता परस्पर सहमतीने] द्वारे केले जातील .
6.	नियमन कायदा: 
हे विक्री करार भारतीय कायद्यांनुसार आणि विशेषतः महाराष्ट्र मुद्रांक कायदा आणि मालमत्ता हस्तांतरण कायदा, १८८२ द्वारे नियंत्रित केले जाईल आणि त्याचा अर्थ लावला जाईल .
________________________________________
साक्षी म्हणून , विक्रेता आणि खरेदीदार यांनी वर लिहिलेल्या पहिल्या दिवशी, महिन्यात आणि वर्षात हे विक्री करार केले आहेत.
स्वाक्षरी केलेले, सील केलेले आणि वितरित केलेले:
विक्रेता: 
(स्वाक्षरी) ___________________नाव: ________________________पत्ता: ____________________
खरेदीदार: 
(स्वाक्षरी) ___________________नाव: ________________________पत्ता: ____________________
साक्षीदार:
1.	नाव: ___________________ स्वाक्षरी: _______________ पत्ता: _______________
2.	नाव: ___________________ स्वाक्षरी: _______________ पत्ता: _______________


"""
    },
    {
        "english_text": """
        SALE DEED
This Sale Deed is executed on this ___ day of ____, 20, at [Place], Maharashtra.
BY AND BETWEEN:
1. [Seller Name], aged about ___ years, residing at [Address], hereinafter referred to as the “SELLER” (which expression shall, unless repugnant to the context, include his/her heirs, legal representatives, executors, administrators, and assigns).
2. [Buyer Name], aged about ___ years, residing at [Address], hereinafter referred to as the “BUYER” (which expression shall, unless repugnant to the context, include his/her heirs, legal representatives, executors, administrators, and assigns).
RECITALS:
A. The SELLER is the absolute and lawful owner of the plot bearing Survey No. [____], admeasuring [____] sq. m./sq. ft., situated at [Village/Town, Taluka, District, Maharashtra] (“Said Property”).
B. The SELLER has clear title to the Said Property, free from all encumbrances, mortgages, liens, charges, or litigation.
C. The SELLER has agreed to sell and the BUYER has agreed to purchase the Said Property for ₹[Amount] (Rupees [Amount in words]).
NOW THIS DEED WITNESSETH AS FOLLOWS:
1.	Sale and Transfer:
The SELLER hereby sells, transfers, and conveys the Said Property to the BUYER absolutely, together with all rights, easements, and appurtenances.
2.	Consideration and Payment:
a. Total consideration of ₹[Amount] has been received by the SELLER from the BUYER by [mode of payment].
b. The receipt of the said amount is hereby acknowledged.
3.	Possession:
Physical and legal possession of the Said Property is delivered to the BUYER on execution of this Deed.
4.	Title and Warranty:
The SELLER represents and warrants that:
o	The Said Property is free from any encumbrances, liens, claims, or litigation.
o	The SELLER has full authority to sell the property.
o	The SELLER shall indemnify the BUYER against any defect in the title or legal claims arising in future.
5.	Taxes, Duties, and Expenses:
a. All stamp duty, registration fees, and other charges for execution of this Deed shall be borne by [Buyer/Seller mutually agreed].
b. All taxes, assessments, and levies up to the date of execution shall be borne by the SELLER.
6.	Covenants:
a. The SELLER shall not interfere with the peaceful possession and use of the property by the BUYER.
b. The BUYER shall have full right to sell, transfer, or mortgage the property after registration.
7.	Governing Law and Jurisdiction:
This Deed shall be governed by and construed under the laws of India, specifically Maharashtra Stamp Act and Transfer of Property Act, 1882. Disputes shall be subject to the exclusive jurisdiction of courts at [City/District].
IN WITNESS WHEREOF, the parties have executed this Sale Deed on the day, month, and year first above written.
SELLER: __________
BUYER: __________
WITNESSES:
1.	Name: ________ Signature: ________
2.	Name: ________ Signature: ________


        """,
        "reference_translation" : """
            विक्री करार
हे विक्री करार २० तारखेला , महाराष्ट्रातील [स्थान] येथे ____ दिवशी अंमलात आणले जाते .
द्वारे आणि दरम्यान:
१. [विक्रेत्याचे नाव] , वय सुमारे ___ वर्षे, [पत्ता] येथे राहणारा , यापुढे "विक्रेता" म्हणून संदर्भित (या अभिव्यक्तीमध्ये, संदर्भाच्या विरुद्ध नसल्यास, त्याचे/तिचे वारस, कायदेशीर प्रतिनिधी, कार्यकारी, प्रशासक आणि नियुक्त केलेले यांचा समावेश असेल).
२. [खरेदीदाराचे नाव] , वय सुमारे ___ वर्षे, [पत्ता] येथे राहणारा , यापुढे "खरेदीदार" म्हणून संदर्भित (या अभिव्यक्तीमध्ये, संदर्भाच्या विरुद्ध नसल्यास, त्याचे/तिचे वारस, कायदेशीर प्रतिनिधी, कार्यकारी, प्रशासक आणि नियुक्त केलेले यांचा समावेश असेल).
मालमत्ता: अ. विक्रेता हा 
[गाव/शहर, तालुका, जिल्हा, महाराष्ट्र] ("वर्तमान मालमत्ता") 
येथे असलेल्या सर्व्हे क्रमांक [____] असलेल्या भूखंडाचा पूर्ण आणि कायदेशीर मालक आहे. ब. विक्रेत्याकडे सदर मालमत्तेवर स्पष्ट मालकी हक्क आहे, जो सर्व भार, गहाणखत, धारणाधिकार, शुल्क किंवा खटल्यापासून मुक्त आहे. क. विक्रेत्याने विक्री करण्यास सहमती दर्शविली आहे आणि खरेदीदाराने सदर मालमत्ता ₹[रक्कम] (रुपये [रक्कम शब्दात]) मध्ये खरेदी करण्यास सहमती दर्शविली आहे .
आता हे कृत्य खालीलप्रमाणे साक्ष देते:
1.	विक्री आणि हस्तांतरण: 
विक्रेता याद्वारे सर्व हक्क, सुविधा आणि उपभोग्य वस्तूंसह, सदर मालमत्ता पूर्णपणे खरेदीदाराला विकतो, हस्तांतरित करतो आणि हस्तांतरित करतो.
2.	विचार आणि देयक: 
अ. विक्रेत्याला खरेदीदाराकडून [देयक पद्धतीद्वारे] एकूण ₹[रक्कम] मिळाली आहे . 
ब. सदर रकमेची पावती याद्वारे पोच दिली जाते.
3.	ताबा: 
या कराराच्या अंमलबजावणीनंतर सदर मालमत्तेचा भौतिक आणि कायदेशीर ताबा खरेदीदाराला दिला जातो.
4.	मालकी हक्क आणि हमी: 
विक्रेता प्रतिनिधित्व करतो आणि हमी देतो की:
o	सदर मालमत्ता कोणत्याही भार, धारणाधिकार, दावे किंवा खटल्यापासून मुक्त आहे.
o	विक्रेत्याला मालमत्ता विकण्याचा पूर्ण अधिकार आहे.
o	भविष्यात उद्भवणाऱ्या मालकी हक्कातील कोणत्याही दोष किंवा कायदेशीर दाव्यांसाठी विक्रेता खरेदीदाराला नुकसानभरपाई देईल.
5.	कर, शुल्क आणि खर्च: 
अ. या कराराच्या अंमलबजावणीसाठी सर्व मुद्रांक शुल्क, नोंदणी शुल्क आणि इतर शुल्क [खरेदीदार/विक्रेता परस्पर सहमतीने] द्वारे वहन केले जातील . 
ब. अंमलबजावणीच्या तारखेपर्यंतचे सर्व कर, कर आकारणी आणि कर विक्रेत्याद्वारे वहन केले जातील.
6.	करार: 
अ. विक्रेत्याने खरेदीदाराच्या मालमत्तेच्या शांततापूर्ण ताब्यामध्ये आणि वापरात हस्तक्षेप करणार नाही . 
ब. नोंदणीनंतर खरेदीदाराला मालमत्ता विकण्याचा, हस्तांतरित करण्याचा किंवा गहाण ठेवण्याचा पूर्ण अधिकार असेल.
7.	नियमन कायदा आणि अधिकारक्षेत्र: 
हे करार भारतीय कायद्यांद्वारे नियंत्रित केले जाईल आणि त्याचा अर्थ लावला जाईल, विशेषतः महाराष्ट्र मुद्रांक कायदा आणि मालमत्ता हस्तांतरण कायदा, १८८२. वाद हे [शहर/जिल्हा] येथील न्यायालयांच्या विशेष अधिकारक्षेत्राच्या अधीन असतील .
साक्षी म्हणून , पक्षांनी वर लिहिलेल्या पहिल्या दिवशी, महिन्यात आणि वर्षात हा विक्री करार अंमलात आणला आहे.
विक्रेता: __________ 
खरेदीदार: __________
साक्षीदार:
1.	नाव: ________ स्वाक्षरी: ________
2.	नाव: ________ स्वाक्षरी: ________

        """
    },
    {
        "english_text" :"""
            SALE DEED
Date: ___ / ___ / 20__
Place: [City], Maharashtra
PARTIES:
•	SELLER: [Name, Address]
•	BUYER: [Name, Address]
WHEREAS:
1.	The SELLER is the absolute owner of Plot No. [], admeasuring [] sq. m./sq. ft., located at [Location].
2.	The BUYER has agreed to purchase the Said Property for ₹[Amount].
3.	The property is free from encumbrances, liens, mortgages, claims, or litigation.
TERMS AND CONDITIONS:
1.	Transfer of Ownership:
The SELLER hereby sells, conveys, and transfers all rights, title, and interest in the Said Property to the BUYER.
2.	Consideration:
The BUYER has paid the full sale consideration of ₹[Amount] to the SELLER, receipt of which is hereby acknowledged.
3.	Possession:
The SELLER delivers physical and legal possession to the BUYER immediately upon execution.
4.	Indemnity:
The SELLER shall indemnify the BUYER against any future claims, disputes, or litigation regarding ownership, boundaries, or encumbrances of the Said Property.
5.	Taxes and Registration:
All costs related to stamp duty, registration fees, and other charges for this Deed shall be borne by the BUYER.
6.	Representations and Warranties:
o	The SELLER has lawful authority to sell the property.
o	The Said Property is not subject to any claims or litigation.
o	No notice from any governmental or municipal authority is pending.
7.	Governing Law and Dispute Resolution:
This Deed is governed by Indian law. Any disputes shall be resolved through arbitration or courts at [City], Maharashtra.
SIGNATURES:
SELLER: ________
BUYER: ________
Witnesses: 1. _______ 2. _______

        """,
        "reference_translation" :"""
            विक्री करार
तारीख: ___ / ___ / २०__ 
ठिकाण: [शहर], महाराष्ट्र
पक्ष:
•	विक्रेता: [नाव, पत्ता]
•	खरेदीदार: [नाव, पत्ता]
जिथे:
1.	विक्रेता हा [स्थान] येथे असलेल्या [ ] चौरस मीटर/चौरस फूट आकाराच्या प्लॉट क्रमांक [] चा पूर्ण मालक आहे.
2.	खरेदीदाराने सदर मालमत्ता ₹[रक्कम] मध्ये खरेदी करण्यास सहमती दर्शविली आहे.
3.	ही मालमत्ता कोणत्याही प्रकारचे भार, धारणाधिकार, गहाणखत, दावे किंवा खटले यापासून मुक्त आहे.
नियम आणि अटी:
1.	मालकीचे हस्तांतरण: 
विक्रेता याद्वारे सदर मालमत्तेतील सर्व हक्क, मालकी हक्क आणि हितसंबंध खरेदीदाराला विकतो, हस्तांतरित करतो आणि हस्तांतरित करतो.
2.	विचार: 
खरेदीदाराने विक्रेत्याला ₹[रक्कम] ची संपूर्ण विक्री मोबदला दिला आहे, ज्याची पावती येथे पोच दिली जाते.
3.	ताबा: 
विक्रेता अंमलबजावणीनंतर लगेचच खरेदीदाराला भौतिक आणि कायदेशीर ताबा देतो.
4.	नुकसानभरपाई: 
विक्रेता खरेदीदाराला भविष्यातील कोणत्याही दाव्यांपासून, वादांपासून किंवा सदर मालमत्तेच्या मालकी, सीमा किंवा भारांबद्दलच्या खटल्यांविरुद्ध नुकसानभरपाई देईल.
5.	कर आणि नोंदणी: 
या करारासाठी मुद्रांक शुल्क, नोंदणी शुल्क आणि इतर शुल्कांशी संबंधित सर्व खर्च खरेदीदाराने वहन करावेत.
6.	प्रतिनिधित्व आणि हमी:
o	विक्रेत्याला मालमत्ता विकण्याचा कायदेशीर अधिकार आहे.
o	सदर मालमत्तेवर कोणताही दावा किंवा खटला चालणार नाही.
o	कोणत्याही सरकारी किंवा महानगरपालिका प्राधिकरणाकडून कोणतीही सूचना प्रलंबित नाही.
7.	नियमन कायदा आणि वाद निराकरण: 
हे करार भारतीय कायद्याद्वारे नियंत्रित केले जाते. कोणतेही वाद मध्यस्थीद्वारे किंवा [शहर], महाराष्ट्र येथील न्यायालयांद्वारे सोडवले जातील.
स्वाक्षऱ्या: 
विक्रेता: ________खरेदीदार: ________साक्षीदार: १. _______ २. _______

        """
    },
    {
        "english_text" :"""
            SALE DEED
Date: ___ / ___ / 20__
Place: [City], Maharashtra
DEFINITIONS:
•	“SELLER” means [Name], legal owner of the property.
•	“BUYER” means [Name], acquiring rights under this Deed.
•	“Said Property” means Plot No. [], measuring [] sq. m./sq. ft., located at [Location].
RECITALS:
1.	The SELLER is the sole owner of the Said Property.
2.	The BUYER has agreed to purchase the Said Property for ₹[Amount].
3.	The Said Property is free from all encumbrances, dues, or litigation.
COVENANTS:
1.	The SELLER transfers all rights, title, and interest in the Said Property to the BUYER.
2.	The SELLER shall indemnify the BUYER against all claims or disputes arising in future.
3.	The BUYER shall have the right to develop, sell, mortgage, or lease the property after registration.
4.	Taxes, municipal dues, and other statutory charges before execution are the responsibility of the SELLER.
5.	Stamp duty, registration fees, and legal costs are borne by the BUYER.
6.	SELLER guarantees peaceful possession of the property to BUYER.
DISPUTE RESOLUTION:
Any dispute arising out of this Deed shall be resolved through arbitration under Indian Arbitration and Conciliation Act, 1996. Courts at [City], Maharashtra shall have jurisdiction.
GOVERNING LAW: Maharashtra and Indian laws.
SIGNATURES:
SELLER: ________
BUYER: ________
Witnesses: 1. ________ 2. ________


        """,
        "reference_translation" :"""
            विक्री करार
तारीख: ___ / ___ / २०__ 
ठिकाण: [शहर], महाराष्ट्र
व्याख्या:
•	"विक्रेता" म्हणजे [नाव], मालमत्तेचा कायदेशीर मालक.
•	"खरेदीदार" म्हणजे [नाव], या करारांतर्गत हक्क मिळवणारा.
•	"उक्त मालमत्ता" म्हणजे [स्थान] येथे स्थित [] चौरस मीटर/चौरस फूट मोजणारा प्लॉट क्रमांक [] .
रिकिटल्स:
1.	विक्रेता हा सेड प्रॉपर्टीचा एकमेव मालक आहे.
2.	खरेदीदाराने सदर मालमत्ता ₹[रक्कम] मध्ये खरेदी करण्यास सहमती दर्शविली आहे.
3.	सदर मालमत्ता सर्व भार, देणी किंवा खटल्यापासून मुक्त आहे.
करार:
1.	विक्रेता सदर मालमत्तेतील सर्व हक्क, मालकी हक्क आणि हितसंबंध खरेदीदाराकडे हस्तांतरित करतो.
2.	भविष्यात उद्भवणाऱ्या सर्व दाव्यांपासून किंवा वादांपासून विक्रेता खरेदीदाराला नुकसानभरपाई देईल.
3.	नोंदणीनंतर खरेदीदाराला मालमत्ता विकसित करण्याचा, विक्री करण्याचा, गहाण ठेवण्याचा किंवा भाडेपट्ट्यावर देण्याचा अधिकार असेल.
4.	अंमलबजावणीपूर्वी कर, महानगरपालिका देयके आणि इतर वैधानिक शुल्क विक्रेत्याची जबाबदारी आहे.
5.	मुद्रांक शुल्क, नोंदणी शुल्क आणि कायदेशीर खर्च खरेदीदाराने उचलले आहेत.
6.	विक्रेता खरेदीदाराला मालमत्तेचा शांततापूर्ण ताबा देण्याची हमी देतो.
वादाचे निराकरण: 
या करारातून उद्भवणारे कोणतेही वाद भारतीय लवाद आणि सामंजस्य कायदा, १९९६ अंतर्गत लवादाद्वारे सोडवले जातील. [शहर], महाराष्ट्र येथील न्यायालयांचे अधिकार क्षेत्र असेल.
शासकीय कायदा: महाराष्ट्र आणि भारतीय कायदे.
स्वाक्षऱ्या: 
विक्रेता: ________खरेदीदार: ________साक्षीदार: १. ________ २. ________


        """
    }
]


prompts = [
    "Translate the following text into Marathi exactly preserving legal terms.",
    "Translate the following text into Marathi in simple, easy-to-understand Marathi.",
    "Translate the following text into Marathi keeping professional/legal tone.",
    """Translate the following English legal text into Marathi exactly preserving all legal terms and clause structures. 
    Use precise legal terminology. Do not simplify or reword any clauses. Maintain numbering, punctuation, and formatting as in the original text.""",
    """Translate the following English legal text into Marathi in a professional and formal legal style suitable for official documentation. 
    Preserve legal terms but make the sentences flow naturally. Use polite, formal Marathi appropriate for contracts, agreements, or deeds.""",
    """Translate the following English legal text into Marathi as a concise but legally accurate summary. Keep all rights, obligations, and 
    clauses intact, but reduce redundancy. Suitable for executives or quick review.""",
    """Translate the following English legal text into Marathi. Ensure all key legal terms like 'possession', 'Seller', 'Buyer', and 'government dues' 
    are correctly translated. Maintain full context and structure. If a term has multiple possible translations, choose the one used in Indian legal documents."""
]

def groq_translate(prompt):
    return client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=[
            {"role": "system", "content": "You are a translator that outputs Marathi text only."},
            {"role": "user", "content": f"{prompt}\n\n{english_text}"}
        ],
        temperature=0.1,
        max_tokens=1500,
    ).choices[0].message.content

def gpt_120b_translate(prompt):
    return client.chat.completions.create(
        model="openai/gpt-oss-120b",
        messages=[
            {"role": "system", "content": "You are a translator that outputs Marathi text only."},
            {"role": "user", "content": f"{prompt}\n\n{english_text}"}
        ],
        temperature=0.1,
        max_tokens=1500,
    ).choices[0].message.content

# ---------------- Run Translations ----------------
results = []

for data in dataset:  # loop over multiple English → Marathi pairs
    english_text = data["english_text"]
    reference_translation = data["reference_translation"]

    for prompt in prompts:  # loop over all prompt styles
        # --- Gemini ---
        gemini_translation = gemini_translate(english_text)
        gemini_translation_pro = gemini_translate_pro(english_text)

        # --- Groq ---
        groq_translation = groq_translate(prompt)

        # --- GPT-OSS ---
        gpt120b_translation = gpt_120b_translate(prompt)

        # --- BLEU Scores ---
        gemini_bleu = sacrebleu.corpus_bleu([gemini_translation], [[reference_translation]]).score
        gemini_pro_bleu = sacrebleu.corpus_bleu([gemini_translation_pro], [[reference_translation]]).score
        groq_bleu = sacrebleu.corpus_bleu([groq_translation], [[reference_translation]]).score
        gpt120b_bleu = sacrebleu.corpus_bleu([gpt120b_translation], [[reference_translation]]).score

        results.append({
            "English Text": english_text[:80] + "...",  # show first 80 chars for readability
            "Prompt": prompt,
            "Google Gemini": gemini_translation,
            "Gemini BLEU": round(gemini_bleu, 2),
            "Google Gemini Pro": gemini_translation_pro,
            "Gemini Pro BLEU": round(gemini_pro_bleu, 2),
            "Groq LLaMA 3.3-70B": groq_translation,
            "Groq BLEU": round(groq_bleu, 2),
            "GPT-OSS 120B": gpt120b_translation,
            "GPT-OSS BLEU": round(gpt120b_bleu, 2),
            "Reference Translation": reference_translation[:80] + "..."  # truncate for table
        })

df = pd.DataFrame(results)
pd.set_option('display.max_colwidth', None)
display(HTML(df.to_html(escape=False)))


>**Bleu Score of each model :** <br>
>**1. PROMPT -** <br>
>> "Translate the following English legal text into Marathi. Ensure all key legal terms like 'possession', 'Seller', 'Buyer', and 'government dues' \n are correctly translated. Maintain full context and structure. If a term has multiple possible translations, choose the one used in Indian legal documents."
* Google Gemini 2.5 flash - 27.50
* Google Gemini 2.0 pro - 31.64
* Groq LLaMA 3.3-70B - 45.37
* GPT-OSS 120B - 25.82
<br>
>**2. PROMPT -** <br>
>> "Translate the following English legal text into Marathi as a concise but legally accurate summary. Keep all rights, obligations, and \n clauses intact, but reduce redundancy. Suitable for executives or quick review."
* Google Gemini 2.5 flash - 27.50
* Google Gemini 2.0 pro - 31.64
* Groq LLaMA 3.3-70B - 45.37
* GPT-OSS 120B - 25.82

Google Gemini	Gemini BLEU	Google Gemini Pro	Gemini Pro BLEU	Groq LLaMA 3.3-70B	Groq BLEU	GPT-OSS 120B	GPT-OSS BLEU

In [16]:
english_text = """
SALE DEED
This Sale Deed is executed on this ___ day of ____, 20 at [Place], Maharashtra.
BY AND BETWEEN
1. [Seller Name], aged about ___ years, residing at [Address], hereinafter referred to as the "SELLER" (which expression shall, unless repugnant to the context or meaning thereof, include his/her heirs, legal representatives, executors, administrators, and assigns).
AND
2. [Buyer Name], aged about ___ years, residing at [Address], hereinafter referred to as the "BUYER" (which expression shall, unless repugnant to the context or meaning thereof, include his/her heirs, legal representatives, executors, administrators, and assigns).
The SELLER and the BUYER are collectively referred to as the “Parties”.
________________________________________
WHEREAS:
1.	The SELLER is the absolute and lawful owner of the plot of land bearing Survey No. [____], admeasuring [____] sq. m./sq. ft., situated at [Village/Town, Taluka, District, Maharashtra], hereinafter referred to as the "Said Property".
2.	The SELLER has agreed to sell the Said Property to the BUYER for a total consideration of ₹[Amount] (Rupees [Amount in words]).
3.	The BUYER has agreed to purchase the Said Property from the SELLER for the said consideration, free from all encumbrances, liens, charges, mortgages, or litigation.
________________________________________
NOW THIS DEED WITNESSETH AS FOLLOWS:
1.	Sale and Transfer:
The SELLER, for the consideration mentioned above, hereby sells, transfers, and conveys the Said Property to the BUYER absolutely, together with all rights, easements, and appurtenances thereto.
2.	Consideration:
The total sale consideration of ₹[Amount] has been received by the SELLER from the BUYER by way of [mode of payment: cheque/cash/bank transfer], the receipt of which the SELLER hereby acknowledges.
3.	Possession:
The SELLER hereby delivers physical and lawful possession of the Said Property to the BUYER on the execution of this Deed.
4.	Title and Warranty:
The SELLER assures that the Said Property is free from all encumbrances, legal disputes, claims, or mortgages. The SELLER further agrees to indemnify the BUYER against any loss arising out of any defect in the title of the Said Property.
5.	Expenses and Registration:
All expenses relating to stamp duty, registration charges, and other incidental costs shall be borne by the [Buyer/Seller mutually agreed].
6.	Governing Law:
This Sale Deed shall be governed by and construed in accordance with the laws of India, and specifically, the Maharashtra Stamp Act and Transfer of Property Act, 1882.
________________________________________
IN WITNESS WHEREOF, the SELLER and the BUYER have executed this Sale Deed on the day, month, and year first above written.
Signed, Sealed, and Delivered by:
SELLER:
(Signature) ___________________
Name: _______________________
Address: ____________________
BUYER:
(Signature) ___________________
Name: _______________________
Address: ____________________
WITNESSES:
1.	Name: ___________________ Signature: _______________ Address: _______________
2.	Name: ___________________ Signature: _______________ Address: _______________

"""
reference_translation = """
विक्री करार
हे विक्री करार २० तारखेला महाराष्ट्रातील [स्थान] येथे ____ दिवशी अंमलात आणले जात आहे .
द्वारे आणि दरम्यान
१. [विक्रेत्याचे नाव] , वय सुमारे ___ वर्षे, [पत्ता] येथे राहणारा , यापुढे "विक्रेता" म्हणून संदर्भित (या अभिव्यक्तीमध्ये, संदर्भ किंवा अर्थाच्या विरुद्ध नसल्यास, त्याचे/तिचे वारस, कायदेशीर प्रतिनिधी, कार्यकारी, प्रशासक आणि नियुक्त केलेले यांचा समावेश असेल).
आणि
२. [खरेदीदाराचे नाव] , वय सुमारे ___ वर्षे, [पत्ता] येथे राहणारा , यापुढे "खरेदीदार" म्हणून संदर्भित (या अभिव्यक्तीमध्ये, संदर्भ किंवा अर्थाच्या विरुद्ध नसल्यास, त्याचे/तिचे वारस, कायदेशीर प्रतिनिधी, कार्यकारी, प्रशासक आणि नियुक्त केलेले यांचा समावेश असेल).
विक्रेता आणि खरेदीदार यांना एकत्रितपणे "पक्ष" असे संबोधले जाते.
________________________________________
जिथे:
1.	[गाव/शहर, तालुका, जिल्हा, महाराष्ट्र] येथे असलेल्या , यापुढे "उक्त मालमत्ता" म्हणून उल्लेख केलेल्या, सर्व्हे क्रमांक [____] असलेल्या [____] चौरस मीटर/चौरस फूट आकाराच्या भूखंडाचा पूर्ण आणि कायदेशीर मालक आहे .
2.	₹[रक्कम] (रुपये [रक्कम शब्दात]) मध्ये विकण्यास सहमती दर्शविली आहे .
3.	खरेदीदाराने विक्रेत्याकडून सदर मालमत्ता सर्व भार, धारणाधिकार, शुल्क, गहाणखत किंवा खटल्यापासून मुक्त करून, सदर मोबदल्यात खरेदी करण्यास सहमती दर्शविली आहे.
________________________________________
आता हे कृत्य खालीलप्रमाणे साक्ष देते:
1.	विक्री आणि हस्तांतरण: 
वर उल्लेख केलेल्या मोबदल्यात, विक्रेता याद्वारे सर्व हक्क, सुविधा आणि उपयोजनांसह, सांगितलेली मालमत्ता पूर्णपणे खरेदीदाराला विकतो, हस्तांतरित करतो आणि हस्तांतरित करतो.
2.	विचार: 
विक्रेत्याला खरेदीदाराकडून [पेमेंटची पद्धत: चेक/रोख/बँक हस्तांतरण] द्वारे एकूण विक्री विचारात घेतलेला ₹[रक्कम] मिळाला आहे , ज्याची पावती विक्रेता याद्वारे मान्य करतो.
3.	ताबा: 
विक्रेता याद्वारे या कराराच्या अंमलबजावणीनंतर खरेदीदाराला उक्त मालमत्तेचा भौतिक आणि कायदेशीर ताबा देतो.
4.	मालकी हक्क आणि हमी: 
विक्रेता खात्री देतो की सदर मालमत्ता सर्व भार, कायदेशीर वाद, दावे किंवा गहाणखतांपासून मुक्त आहे. विक्रेता सदर मालमत्तेच्या मालकी हक्कातील कोणत्याही दोषामुळे उद्भवणाऱ्या कोणत्याही नुकसानाविरुद्ध खरेदीदाराला भरपाई देण्यास सहमत आहे.
5.	खर्च आणि नोंदणी: 
मुद्रांक शुल्क, नोंदणी शुल्क आणि इतर आनुषंगिक खर्चाशी संबंधित सर्व खर्च [खरेदीदार/विक्रेता परस्पर सहमतीने] द्वारे केले जातील .
6.	नियमन कायदा: 
हे विक्री करार भारतीय कायद्यांनुसार आणि विशेषतः महाराष्ट्र मुद्रांक कायदा आणि मालमत्ता हस्तांतरण कायदा, १८८२ द्वारे नियंत्रित केले जाईल आणि त्याचा अर्थ लावला जाईल .
________________________________________
साक्षी म्हणून , विक्रेता आणि खरेदीदार यांनी वर लिहिलेल्या पहिल्या दिवशी, महिन्यात आणि वर्षात हे विक्री करार केले आहेत.
स्वाक्षरी केलेले, सील केलेले आणि वितरित केलेले:
विक्रेता: 
(स्वाक्षरी) ___________________नाव: ________________________पत्ता: ____________________
खरेदीदार: 
(स्वाक्षरी) ___________________नाव: ________________________पत्ता: ____________________
साक्षीदार:
1.	नाव: ___________________ स्वाक्षरी: _______________ पत्ता: _______________
2.	नाव: ___________________ स्वाक्षरी: _______________ पत्ता: _______________


"""

In [4]:
prompts = [
    "Translate the following text into Marathi exactly preserving legal terms.",
    "Translate the following text into Marathi in simple, easy-to-understand Marathi.",
    "Translate the following text into Marathi keeping professional/legal tone.",
    """Translate the following English legal text into Marathi exactly preserving all legal terms and clause structures. 
    Use precise legal terminology. Do not simplify or reword any clauses. Maintain numbering, punctuation, and formatting as in the original text.""",
    """Translate the following English legal text into Marathi as a concise but legally accurate summary. Keep all rights, obligations, and 
    clauses intact, but reduce redundancy. Suitable for executives or quick review.""",
    """Translate the following English legal text into Marathi. Ensure all key legal terms like 'possession', 'Seller', 'Buyer', and 'government dues' 
    are correctly translated. Maintain full context and structure. If a term has multiple possible translations, choose the one used in Indian legal documents."""
]

In [9]:
best_results = []

for row in results:
    # Extract only BLEU scores
    bleu_scores = {
        # "HF BLEU": row.get("HF BLEU", -1),
        "Gemini BLEU": row.get("Gemini BLEU", -1),
        "Gemini Pro BLEU": row.get("Gemini Pro BLEU", -1),
        "Groq BLEU": row.get("Groq BLEU", -1),
        "GPT-OSS BLEU": row.get("GPT-OSS BLEU", -1),
        # "M2M100 BLEU": row.get("M2M100 BLEU", -1)
    }
    
    # Pick highest scoring model
    best_model = max(bleu_scores, key=bleu_scores.get)
    best_score = bleu_scores[best_model]
    
    # Map back to actual translation text column
    model_name_map = {
        # "HF BLEU": "HuggingFace MarianMT",
        "Gemini BLEU": "Google Gemini",
        "Gemini Pro BLEU": "Google Gemini Pro",
        "Groq BLEU": "Groq LLaMA 3.3-70B",
        "GPT-OSS BLEU": "GPT-OSS 120B",
        # "M2M100 BLEU": "M2M100"
    }
    
    best_translation = row[model_name_map[best_model]]
    
    best_results.append({
        "Prompt": row["Prompt"],
        "Best Model": model_name_map[best_model],
        "Best BLEU": best_score,
        "Best Translation": best_translation
    })


In [ ]:
import pandas as pd
# from googletrans import Translator
from groq import Groq
from IPython.display import display, HTML
import sacrebleu


english_text = """
SALE DEED
This Sale Deed is executed on this ___ day of ____, 20 at [Place], Maharashtra.
BY AND BETWEEN
1. [Seller Name], aged about ___ years, residing at [Address], hereinafter referred to as the "SELLER" (which expression shall, unless repugnant to the context or meaning thereof, include his/her heirs, legal representatives, executors, administrators, and assigns).
AND
2. [Buyer Name], aged about ___ years, residing at [Address], hereinafter referred to as the "BUYER" (which expression shall, unless repugnant to the context or meaning thereof, include his/her heirs, legal representatives, executors, administrators, and assigns).
The SELLER and the BUYER are collectively referred to as the “Parties”.
________________________________________
WHEREAS:
1.	The SELLER is the absolute and lawful owner of the plot of land bearing Survey No. [____], admeasuring [____] sq. m./sq. ft., situated at [Village/Town, Taluka, District, Maharashtra], hereinafter referred to as the "Said Property".
2.	The SELLER has agreed to sell the Said Property to the BUYER for a total consideration of ₹[Amount] (Rupees [Amount in words]).
3.	The BUYER has agreed to purchase the Said Property from the SELLER for the said consideration, free from all encumbrances, liens, charges, mortgages, or litigation.
________________________________________
NOW THIS DEED WITNESSETH AS FOLLOWS:
1.	Sale and Transfer:
The SELLER, for the consideration mentioned above, hereby sells, transfers, and conveys the Said Property to the BUYER absolutely, together with all rights, easements, and appurtenances thereto.
2.	Consideration:
The total sale consideration of ₹[Amount] has been received by the SELLER from the BUYER by way of [mode of payment: cheque/cash/bank transfer], the receipt of which the SELLER hereby acknowledges.
3.	Possession:
The SELLER hereby delivers physical and lawful possession of the Said Property to the BUYER on the execution of this Deed.
4.	Title and Warranty:
The SELLER assures that the Said Property is free from all encumbrances, legal disputes, claims, or mortgages. The SELLER further agrees to indemnify the BUYER against any loss arising out of any defect in the title of the Said Property.
5.	Expenses and Registration:
All expenses relating to stamp duty, registration charges, and other incidental costs shall be borne by the [Buyer/Seller mutually agreed].
6.	Governing Law:
This Sale Deed shall be governed by and construed in accordance with the laws of India, and specifically, the Maharashtra Stamp Act and Transfer of Property Act, 1882.
________________________________________
IN WITNESS WHEREOF, the SELLER and the BUYER have executed this Sale Deed on the day, month, and year first above written.
Signed, Sealed, and Delivered by:
SELLER:
(Signature) ___________________
Name: _______________________
Address: ____________________
BUYER:
(Signature) ___________________
Name: _______________________
Address: ____________________
WITNESSES:
1.	Name: ___________________ Signature: _______________ Address: _______________
2.	Name: ___________________ Signature: _______________ Address: _______________

"""

# Reference translation (choose one carefully – e.g. legal Marathi translation you trust most)
reference_translation = """विक्री करार
हे विक्री करार २० तारखेला महाराष्ट्रातील [स्थान] येथे ____ दिवशी अंमलात आणले जात आहे .
द्वारे आणि दरम्यान
१. [विक्रेत्याचे नाव] , वय सुमारे ___ वर्षे, [पत्ता] येथे राहणारा , यापुढे "विक्रेता" म्हणून संदर्भित (या अभिव्यक्तीमध्ये, संदर्भ किंवा अर्थाच्या विरुद्ध नसल्यास, त्याचे/तिचे वारस, कायदेशीर प्रतिनिधी, कार्यकारी, प्रशासक आणि नियुक्त केलेले यांचा समावेश असेल).
आणि
२. [खरेदीदाराचे नाव] , वय सुमारे ___ वर्षे, [पत्ता] येथे राहणारा , यापुढे "खरेदीदार" म्हणून संदर्भित (या अभिव्यक्तीमध्ये, संदर्भ किंवा अर्थाच्या विरुद्ध नसल्यास, त्याचे/तिचे वारस, कायदेशीर प्रतिनिधी, कार्यकारी, प्रशासक आणि नियुक्त केलेले यांचा समावेश असेल).
विक्रेता आणि खरेदीदार यांना एकत्रितपणे "पक्ष" असे संबोधले जाते.
________________________________________
जिथे:
1.	[गाव/शहर, तालुका, जिल्हा, महाराष्ट्र] येथे असलेल्या , यापुढे "उक्त मालमत्ता" म्हणून उल्लेख केलेल्या, सर्व्हे क्रमांक [____] असलेल्या [____] चौरस मीटर/चौरस फूट आकाराच्या भूखंडाचा पूर्ण आणि कायदेशीर मालक आहे .
2.	₹[रक्कम] (रुपये [रक्कम शब्दात]) मध्ये विकण्यास सहमती दर्शविली आहे .
3.	खरेदीदाराने विक्रेत्याकडून सदर मालमत्ता सर्व भार, धारणाधिकार, शुल्क, गहाणखत किंवा खटल्यापासून मुक्त करून, सदर मोबदल्यात खरेदी करण्यास सहमती दर्शविली आहे.
________________________________________
आता हे कृत्य खालीलप्रमाणे साक्ष देते:
1.	विक्री आणि हस्तांतरण: 
वर उल्लेख केलेल्या मोबदल्यात, विक्रेता याद्वारे सर्व हक्क, सुविधा आणि उपयोजनांसह, सांगितलेली मालमत्ता पूर्णपणे खरेदीदाराला विकतो, हस्तांतरित करतो आणि हस्तांतरित करतो.
2.	विचार: 
विक्रेत्याला खरेदीदाराकडून [पेमेंटची पद्धत: चेक/रोख/बँक हस्तांतरण] द्वारे एकूण विक्री विचारात घेतलेला ₹[रक्कम] मिळाला आहे , ज्याची पावती विक्रेता याद्वारे मान्य करतो.
3.	ताबा: 
विक्रेता याद्वारे या कराराच्या अंमलबजावणीनंतर खरेदीदाराला उक्त मालमत्तेचा भौतिक आणि कायदेशीर ताबा देतो.
4.	मालकी हक्क आणि हमी: 
विक्रेता खात्री देतो की सदर मालमत्ता सर्व भार, कायदेशीर वाद, दावे किंवा गहाणखतांपासून मुक्त आहे. विक्रेता सदर मालमत्तेच्या मालकी हक्कातील कोणत्याही दोषामुळे उद्भवणाऱ्या कोणत्याही नुकसानाविरुद्ध खरेदीदाराला भरपाई देण्यास सहमत आहे.
5.	खर्च आणि नोंदणी: 
मुद्रांक शुल्क, नोंदणी शुल्क आणि इतर आनुषंगिक खर्चाशी संबंधित सर्व खर्च [खरेदीदार/विक्रेता परस्पर सहमतीने] द्वारे केले जातील .
6.	नियमन कायदा: 
हे विक्री करार भारतीय कायद्यांनुसार आणि विशेषतः महाराष्ट्र मुद्रांक कायदा आणि मालमत्ता हस्तांतरण कायदा, १८८२ द्वारे नियंत्रित केले जाईल आणि त्याचा अर्थ लावला जाईल .
________________________________________
साक्षी म्हणून , विक्रेता आणि खरेदीदार यांनी वर लिहिलेल्या पहिल्या दिवशी, महिन्यात आणि वर्षात हे विक्री करार केले आहेत.
स्वाक्षरी केलेले, सील केलेले आणि वितरित केलेले:
विक्रेता: 
(स्वाक्षरी) ___________________नाव: ________________________पत्ता: ____________________
खरेदीदार: 
(स्वाक्षरी) ___________________नाव: ________________________पत्ता: ____________________
साक्षीदार:
1.	नाव: ___________________ स्वाक्षरी: _______________ पत्ता: _______________
2.	नाव: ___________________ स्वाक्षरी: _______________ पत्ता: _______________


"""


prompts = [
    "Translate the following text into Marathi exactly preserving legal terms.",
    "Translate the following text into Marathi in simple, easy-to-understand Marathi.",
    "Translate the following text into Marathi keeping professional/legal tone.",
    """Translate the following English legal text into Marathi exactly preserving all legal terms and clause structures. 
    Use precise legal terminology. Do not simplify or reword any clauses. Maintain numbering, punctuation, and formatting as in the original text.""",
    """Translate the following English legal text into Marathi in a professional and formal legal style suitable for official documentation. 
    Preserve legal terms but make the sentences flow naturally. Use polite, formal Marathi appropriate for contracts, agreements, or deeds.""",
    """Translate the following English legal text into Marathi as a concise but legally accurate summary. Keep all rights, obligations, and 
    clauses intact, but reduce redundancy. Suitable for executives or quick review.""",
    """Translate the following English legal text into Marathi. Ensure all key legal terms like 'possession', 'Seller', 'Buyer', and 'government dues' 
    are correctly translated. Maintain full context and structure. If a term has multiple possible translations, choose the one used in Indian legal documents."""

]

def groq_translate(prompt):
    return client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=[
            {"role": "system", "content": "You are a translator that outputs Marathi text only."},
            {"role": "user", "content": f"{prompt}\n\n{english_text}"}
        ],
        temperature=0.1,
        max_tokens=1500,
    ).choices[0].message.content

def gpt_120b_translate(prompt):
    return client.chat.completions.create(
        model="openai/gpt-oss-120b",
        messages=[
            {"role": "system", "content": "You are a translator that outputs Marathi text only."},
            {"role": "user", "content": f"{prompt}\n\n{english_text}"}
        ],
        temperature=0.1,
        max_tokens=1500,
    ).choices[0].message.content

results = []

for prompt in prompts:
    # Google Translate
    # google_translation = translator.translate(english_text, dest='mr').text

    # Groq
    groq_translation = groq_translate(prompt)

    # GPT-OSS 120B
    gpt120b_translation = gpt_120b_translate(prompt)

    # ✅ BLEU Scores
    # google_bleu = sacrebleu.corpus_bleu([google_translation], [[reference_translation]]).score
    groq_bleu = sacrebleu.corpus_bleu([groq_translation], [[reference_translation]]).score
    gpt120b_bleu = sacrebleu.corpus_bleu([gpt120b_translation], [[reference_translation]]).score

    results.append({
        "Prompt": prompt,
        # "Google Translate": google_translation,
        # "Google BLEU": round(google_bleu, 2),
        "Groq LLaMA 3.3-70B": groq_translation,
        "Groq BLEU": round(groq_bleu, 2),
        "GPT-OSS 120B": gpt120b_translation,
        "GPT-OSS BLEU": round(gpt120b_bleu, 2)
    })

df = pd.DataFrame(results)
pd.set_option('display.max_colwidth', None)
display(HTML(df.to_html(escape=False)))


In [10]:
df_best = pd.DataFrame(best_results)
pd.set_option('display.max_colwidth', None)
display(HTML(df_best.to_html(escape=False)))


,Prompt,Best Model,Best BLEU,Best Translation
0,Translate the following text into Marathi exactly preserving legal terms.,Groq LLaMA 3.3-70B,60.72,"विक्री करार\nहा विक्री करार खालील ___ दिनांक ____, २० या दिवशी [स्थान], महाराष्ट्र येथे केला आहे.\nआणि यामध्ये खालीलप्रमाणे करार केला आहे:\n१. [विक्रेत्याचे नाव], वय सुमारे ___ वर्षे, राहिवाशी [पत्ता], यांना यापुढे ""विक्रेता"" (ज्याचा अर्थ विक्रेत्याचे वारस, कायदेशीर प्रतिनिधी, एक्झिक्यूटर, अॅडमिनिस्ट्रेटर आणि असाईन्स यांचा समावेश असेल) म्हणून संबोधले जाईल.\nआणि\n२. [खरेददाराचे नाव], वय सुमारे ___ वर्षे, राहिवाशी [पत्ता], यांना यापुढे ""खरेददार"" (ज्याचा अर्थ खरेददाराचे वारस, कायदेशीर प्रतिनिधी, एक्झिक्यूटर, अॅडमिनिस्ट्रेटर आणि असाईन्स यांचा समावेश असेल) म्हणून संबोधले जाईल.\nविक्रेता आणि खरेददार यांना सामूहिकपणे ""पक्ष"" म्हणून संबोधले जाईल.\n________________________________________\nयासाठी कि:\n१. विक्रेता हे सर्व्हे नंबर [____] असलेल्या जागेचे निरपेक्ष आणि कायदेशीर मालक आहेत, ज्याचे क्षेत्रफळ [____] चौरस मी. / चौरस फूट आहे, जी [गाव/शहर, तालुका, जिल्हा, महाराष्ट्र] येथे स्थित आहे, याला यापुढे ""सांगितलेली मालमत्ता"" म्हणून संबोधले जाईल.\n२. विक्रेत्याने सांगितलेली मालमत्ता खरेददाराला एकूण ₹[रकम] (रुपये [रकमेचे शब्द]) या मोबदल्यासाठी विकण्यास सहमती दिली आहे.\n३. खरेददाराने विक्रेत्याकडून सांगितलेली मालमत्ता त्या मोबदल्यासाठी विकत घेण्यास सहमती दिली आहे, जी सर्व भार, लiens, चार्जेस, मॉर्गेजेस किंवा खटल्यांपासून मुक्त आहे.\n________________________________________\nअता हा करार खालीलप्रमाणे साक्ष देतो:\n१. विक्री आणि हस्तांतरण:\nविक्रेत्याने वर नमूद केलेल्या मोबदल्यासाठी सांगितलेली मालमत्ता खरेददाराला निरपेक्षपणे विकून दिली आहे, त्यासह सर्व हक्क, ईजमेंट्स आणि अपुर्टनन्सेस.\n२. मोबदला:\nएकूण विक्री मोबदला ₹[रकम] विक्रेत्याला खरेददाराकडून [चेक/कॅश/बँक हस्तांतरण] या मार्गाने मिळाला आहे, ज्याची पावती विक्रेत्याने दिली आहे.\n३. ताबा:\nविक्रेत्याने सांगितलेली मालमत्ता या कराराच्या निर्वाहाने खरेददाराला भौतिक आणि कायदेशीर ताबा दिला आहे.\n४. मालकी आणि हमी:\nविक्रेत्याने सांगितलेली मालमत्ता सर्व भार, कायदेशीर विवाद, दावे किंवा मॉर्गेजेसपासून मुक्त आहे याची खात्री दिली आहे. विक्रेत्याने आणखी सांगितलेल्या मालमत्तेच्या मालकीहक्कातील कोणत्याही दोषामुळे होणारे नुकसान भरपाई देण्यास सहमती दिली आहे.\n५. खर्च आणि नोंदणी:\nस्टॅम्प ड्यूटी, नोंदणी शुल्क आणि इतर अप्रत्यक्ष खर्च यांचा सर्व खर्च [खरेददार/विक्रेता दोघांनी सहमती दिली आहे] यांनी सहन केला पाहिजे.\n६. कायदेशीर अधिकार:\nहा विक्री करार भारताच्या कायद्यांनुसार आणि विशेषतः महाराष्ट्र स्टॅम्प अधिनियम आणि संपत्ती हस्तांतरण अधिनियम, १८८२ अनुसार नियंत्रित केला जाईल.\n________________________________________\nयाची साक्ष आहे की, विक्रेता आणि खरेददार यांनी वर नमूद केलेल्या दिनांक, महिना आणि वर्षी हा विक्री करार केला आहे.\nस्वाक्षरी, सील आणि वितरित केले आहे:\nविक्रेता:\n(स्वाक्षरी) ___________________\nनाव: _______________________\nपत्ता: ____________________\nखरेददार:\n(स्वाक्षरी) ___________________\nनाव: _______________________\nपत्ता: ____________________\nसाक्षीदार:\n१. नाव: ___________________ स्वाक्षरी: _______________ पत्ता: _______________\n२. नाव: ___________________ स्वाक्षरी: _______________ पत्ता: _______________"
1,"Translate the following text into Marathi in simple, easy-to-understand Marathi.",Groq LLaMA 3.3-70B,62.38,"विक्री करार\nहा विक्री करार या दिनांक ___ ___ 20 ___ रोजी [स्थळ], महाराष्ट्र येथे केला आहे.\nआणि यामध्ये खालीलप्रमाणे करार केला आहे:\n1. [विक्रेत्याचे नाव], वय ___ वर्षे, सध्या राहिवाश [पत्ता] यांना यापुढे ""विक्रेता"" असे संबोधले जाईल (ज्यामध्ये त्यांचे वारस, कायदेशीर प्रतिनिधी, कार्यकारी, व्यवस्थापक आणि नियुक्तीकर्ते यांचा समावेश आहे).\nआणि\n2. [गिरणाराचे नाव], वय ___ वर्षे, सध्या राहिवाश [पत्ता] यांना यापुढे ""गिरणारा"" असे संबोधले जाईल (ज्यामध्ये त्यांचे वारस, कायदेशीर प्रतिनिधी, कार्यकारी, व्यवस्थापक आणि नियुक्तीकर्ते यांचा समावेश आहे).\nविक्रेता आणि गिरणारा यांना सामूहिकपणे ""पक्ष"" असे संबोधले जाईल.\n________________________________________\nयासाठी कारण असे आहे की:\n1. विक्रेता हे भूखंड क्रमांक [____] असलेल्या जागेचा निरपेक्ष आणि कायदेशीर मालक आहे, ज्याचे क्षेत्रफळ [____] चौरस मी. / चौरस फूट आहे, जी [गाव/शहर, तालुका, 

> **BLEU Score :**
>>1. **Prompt :** Translate the following English legal text into Marathi exactly preserving all legal terms and clause structures. \n Use precise legal terminology. Do not simplify or reword any clauses. Maintain numbering, punctuation, and formatting as in the original text.<br>
**Model : Groq LLaMA 3.3-70B   Score - 63.60**<br>2. **Prompt :** Translate the following English legal text into Marathi in a professional and formal legal style suitable for official documentation. \n Preserve legal terms but make the sentences flow naturally. Use polite, formal Marathi appropriate for contracts, agreements, or deeds<br>
**Model : Google Gemini 2.0 Pro   Score - 59.32**<br>3. **Prompt :** Translate the following text into Marathi keeping professional/legal tone.<br>
**Model : GPT-OSS 120B   Score - 59.80**<br><br>

> *i have selected top score from these three models because with other prompts these models are scoring < 50* 

In [ ]:
import torch
#from transformers import MarianMTModel, MarianTokenizer
import pandas as pd
# from googletrans import Translator
from groq import Groq
from IPython.display import display, HTML
import sacrebleu
from google import genai  # Gemini
from google.genai import types
    
def translate_en_to_mr(text: str) -> str:
    inputs = tokenizer([text], return_tensors="pt", padding=True, truncation=True).to(device)
    translated_tokens = model.generate(**inputs, max_length=128)
    translated_text = tokenizer.decode(translated_tokens[0], skip_special_tokens=True)
    return translated_text

# ---------------- Gemini Client ----------------
gemini_client = genai.Client(api_key="GEMINI_API_KEY")

def gemini_translate(text: str) -> str:
    response = gemini_client.models.generate_content(
        model="gemini-2.0-flash",
        contents=f"""Translate the following English legal text into Marathi in a professional and formal legal style suitable for official documentation. 
    Preserve legal terms but make the sentences flow naturally. Use polite, formal Marathi appropriate for contracts, agreements, or deeds.\n\n{text}""",
        config=types.GenerateContentConfig(
            temperature=0.0  # deterministic output
        )
    )
    return response.text

def gemini_translate_pro(text: str) -> str:
    response = gemini_client.models.generate_content(
        model="gemini-2.5-pro",   # 🔄 switched to 2.5 Pro
        contents=f"""Translate the following English legal text into Marathi in a professional and formal legal style suitable for official documentation. 
    Preserve legal terms but make the sentences flow naturally. Use polite, formal Marathi appropriate for contracts, agreements, or deeds.\n\n{text}""",
       config=types.GenerateContentConfig(
            temperature=0.0  # deterministic output
        )     
    )
    return response.text


# ---------------- Groq Client ----------------
client = Groq(api_key="GROQ_API_KEY")

english_text = """
SALE DEED
This Sale Deed is executed on this ___ day of ____, 20 at [Place], Maharashtra.
BY AND BETWEEN
1. [Seller Name], aged about ___ years, residing at [Address], hereinafter referred to as the "SELLER" (which expression shall, unless repugnant to the context or meaning thereof, include his/her heirs, legal representatives, executors, administrators, and assigns).
AND
2. [Buyer Name], aged about ___ years, residing at [Address], hereinafter referred to as the "BUYER" (which expression shall, unless repugnant to the context or meaning thereof, include his/her heirs, legal representatives, executors, administrators, and assigns).
The SELLER and the BUYER are collectively referred to as the “Parties”.
________________________________________
WHEREAS:
1.	The SELLER is the absolute and lawful owner of the plot of land bearing Survey No. [____], admeasuring [____] sq. m./sq. ft., situated at [Village/Town, Taluka, District, Maharashtra], hereinafter referred to as the "Said Property".
2.	The SELLER has agreed to sell the Said Property to the BUYER for a total consideration of ₹[Amount] (Rupees [Amount in words]).
3.	The BUYER has agreed to purchase the Said Property from the SELLER for the said consideration, free from all encumbrances, liens, charges, mortgages, or litigation.
________________________________________
NOW THIS DEED WITNESSETH AS FOLLOWS:
1.	Sale and Transfer:
The SELLER, for the consideration mentioned above, hereby sells, transfers, and conveys the Said Property to the BUYER absolutely, together with all rights, easements, and appurtenances thereto.
2.	Consideration:
The total sale consideration of ₹[Amount] has been received by the SELLER from the BUYER by way of [mode of payment: cheque/cash/bank transfer], the receipt of which the SELLER hereby acknowledges.
3.	Possession:
The SELLER hereby delivers physical and lawful possession of the Said Property to the BUYER on the execution of this Deed.
4.	Title and Warranty:
The SELLER assures that the Said Property is free from all encumbrances, legal disputes, claims, or mortgages. The SELLER further agrees to indemnify the BUYER against any loss arising out of any defect in the title of the Said Property.
5.	Expenses and Registration:
All expenses relating to stamp duty, registration charges, and other incidental costs shall be borne by the [Buyer/Seller mutually agreed].
6.	Governing Law:
This Sale Deed shall be governed by and construed in accordance with the laws of India, and specifically, the Maharashtra Stamp Act and Transfer of Property Act, 1882.
________________________________________
IN WITNESS WHEREOF, the SELLER and the BUYER have executed this Sale Deed on the day, month, and year first above written.
Signed, Sealed, and Delivered by:
SELLER:
(Signature) ___________________
Name: _______________________
Address: ____________________
BUYER:
(Signature) ___________________
Name: _______________________
Address: ____________________
WITNESSES:
1.	Name: ___________________ Signature: _______________ Address: _______________
2.	Name: ___________________ Signature: _______________ Address: _______________

"""
reference_translation = """विक्री करार
हे विक्री करार २० तारखेला महाराष्ट्रातील [स्थान] येथे ____ दिवशी अंमलात आणले जात आहे .
द्वारे आणि दरम्यान
१. [विक्रेत्याचे नाव] , वय सुमारे ___ वर्षे, [पत्ता] येथे राहणारा , यापुढे "विक्रेता" म्हणून संदर्भित (या अभिव्यक्तीमध्ये, संदर्भ किंवा अर्थाच्या विरुद्ध नसल्यास, त्याचे/तिचे वारस, कायदेशीर प्रतिनिधी, कार्यकारी, प्रशासक आणि नियुक्त केलेले यांचा समावेश असेल).
आणि
२. [खरेदीदाराचे नाव] , वय सुमारे ___ वर्षे, [पत्ता] येथे राहणारा , यापुढे "खरेदीदार" म्हणून संदर्भित (या अभिव्यक्तीमध्ये, संदर्भ किंवा अर्थाच्या विरुद्ध नसल्यास, त्याचे/तिचे वारस, कायदेशीर प्रतिनिधी, कार्यकारी, प्रशासक आणि नियुक्त केलेले यांचा समावेश असेल).
विक्रेता आणि खरेदीदार यांना एकत्रितपणे "पक्ष" असे संबोधले जाते.
________________________________________
जिथे:
1.	[गाव/शहर, तालुका, जिल्हा, महाराष्ट्र] येथे असलेल्या , यापुढे "उक्त मालमत्ता" म्हणून उल्लेख केलेल्या, सर्व्हे क्रमांक [____] असलेल्या [____] चौरस मीटर/चौरस फूट आकाराच्या भूखंडाचा पूर्ण आणि कायदेशीर मालक आहे .
2.	₹[रक्कम] (रुपये [रक्कम शब्दात]) मध्ये विकण्यास सहमती दर्शविली आहे .
3.	खरेदीदाराने विक्रेत्याकडून सदर मालमत्ता सर्व भार, धारणाधिकार, शुल्क, गहाणखत किंवा खटल्यापासून मुक्त करून, सदर मोबदल्यात खरेदी करण्यास सहमती दर्शविली आहे.
________________________________________
आता हे कृत्य खालीलप्रमाणे साक्ष देते:
1.	विक्री आणि हस्तांतरण: 
वर उल्लेख केलेल्या मोबदल्यात, विक्रेता याद्वारे सर्व हक्क, सुविधा आणि उपयोजनांसह, सांगितलेली मालमत्ता पूर्णपणे खरेदीदाराला विकतो, हस्तांतरित करतो आणि हस्तांतरित करतो.
2.	विचार: 
विक्रेत्याला खरेदीदाराकडून [पेमेंटची पद्धत: चेक/रोख/बँक हस्तांतरण] द्वारे एकूण विक्री विचारात घेतलेला ₹[रक्कम] मिळाला आहे , ज्याची पावती विक्रेता याद्वारे मान्य करतो.
3.	ताबा: 
विक्रेता याद्वारे या कराराच्या अंमलबजावणीनंतर खरेदीदाराला उक्त मालमत्तेचा भौतिक आणि कायदेशीर ताबा देतो.
4.	मालकी हक्क आणि हमी: 
विक्रेता खात्री देतो की सदर मालमत्ता सर्व भार, कायदेशीर वाद, दावे किंवा गहाणखतांपासून मुक्त आहे. विक्रेता सदर मालमत्तेच्या मालकी हक्कातील कोणत्याही दोषामुळे उद्भवणाऱ्या कोणत्याही नुकसानाविरुद्ध खरेदीदाराला भरपाई देण्यास सहमत आहे.
5.	खर्च आणि नोंदणी: 
मुद्रांक शुल्क, नोंदणी शुल्क आणि इतर आनुषंगिक खर्चाशी संबंधित सर्व खर्च [खरेदीदार/विक्रेता परस्पर सहमतीने] द्वारे केले जातील .
6.	नियमन कायदा: 
हे विक्री करार भारतीय कायद्यांनुसार आणि विशेषतः महाराष्ट्र मुद्रांक कायदा आणि मालमत्ता हस्तांतरण कायदा, १८८२ द्वारे नियंत्रित केले जाईल आणि त्याचा अर्थ लावला जाईल .
________________________________________
साक्षी म्हणून , विक्रेता आणि खरेदीदार यांनी वर लिहिलेल्या पहिल्या दिवशी, महिन्यात आणि वर्षात हे विक्री करार केले आहेत.
स्वाक्षरी केलेले, सील केलेले आणि वितरित केलेले:
विक्रेता: 
(स्वाक्षरी) ___________________नाव: ________________________पत्ता: ____________________
खरेदीदार: 
(स्वाक्षरी) ___________________नाव: ________________________पत्ता: ____________________
साक्षीदार:
1.	नाव: ___________________ स्वाक्षरी: _______________ पत्ता: _______________
2.	नाव: ___________________ स्वाक्षरी: _______________ पत्ता: _______________


"""


prompts = [
    "Translate the following text into Marathi exactly preserving legal terms.",
    "Translate the following text into Marathi in simple, easy-to-understand Marathi.",
    "Translate the following text into Marathi keeping professional/legal tone.",
    """Translate the following English legal text into Marathi exactly preserving all legal terms and clause structures. 
    Use precise legal terminology. Do not simplify or reword any clauses. Maintain numbering, punctuation, and formatting as in the original text.""",
    """Translate the following English legal text into Marathi as a concise but legally accurate summary. Keep all rights, obligations, and 
    clauses intact, but reduce redundancy. Suitable for executives or quick review.""",
    """Translate the following English legal text into Marathi. Ensure all key legal terms like 'possession', 'Seller', 'Buyer', and 'government dues' 
    are correctly translated. Maintain full context and structure. If a term has multiple possible translations, choose the one used in Indian legal documents."""
]

def groq_translate(prompt):
    return client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=[
            {"role": "system", "content": """Translate the following English legal text into Marathi in a professional and formal legal style suitable for official documentation. 
    Preserve legal terms but make the sentences flow naturally. Use polite, formal Marathi appropriate for contracts, agreements, or deeds."""},
            {"role": "user", "content": f"{prompt}\n\n{english_text}"}
        ],
        temperature=0.0,
        max_tokens=1500,
    ).choices[0].message.content

def gpt_120b_translate(prompt):
    return client.chat.completions.create(
        model="openai/gpt-oss-120b",
        messages=[
            {"role": "system", "content": """Translate the following English legal text into Marathi in a professional and formal legal style suitable for official documentation. 
    Preserve legal terms but make the sentences flow naturally. Use polite, formal Marathi appropriate for contracts, agreements, or deeds."""},
            {"role": "user", "content": f"{prompt}\n\n{english_text}"}
        ],
        temperature=0.0,
        max_tokens=1500,
    ).choices[0].message.content

# ---------------- Run Translations ----------------
results = []

for prompt in prompts:  # ✅ only loop through prompts
    # --- Gemini ---
    gemini_translation = gemini_translate(f"{prompt}\n\n{english_text}")
    gemini_translation_pro = gemini_translate_pro(f"{prompt}\n\n{english_text}")

    # --- Groq ---
    groq_translation = groq_translate(prompt)

    # --- GPT-OSS ---
    gpt120b_translation = gpt_120b_translate(prompt)

    # --- BLEU Scores ---
    gemini_bleu = sacrebleu.corpus_bleu([gemini_translation], [[reference_translation]]).score
    gemini_pro_bleu = sacrebleu.corpus_bleu([gemini_translation_pro], [[reference_translation]]).score
    groq_bleu = sacrebleu.corpus_bleu([groq_translation], [[reference_translation]]).score
    gpt120b_bleu = sacrebleu.corpus_bleu([gpt120b_translation], [[reference_translation]]).score

    results.append({
        "English Text": english_text[:100] + "...",
        "Prompt": prompt,
        "Google Gemini": gemini_translation,
        "Gemini BLEU": round(gemini_bleu, 2),
        "Google Gemini Pro": gemini_translation_pro,
        "Gemini Pro BLEU": round(gemini_pro_bleu, 2),
        "Groq LLaMA 3.3-70B": groq_translation,
        "Groq BLEU": round(groq_bleu, 2),
        "GPT-OSS 120B": gpt120b_translation,
        "GPT-OSS BLEU": round(gpt120b_bleu, 2),
        "Reference Translation": reference_translation[:100] + "..."
    })

df = pd.DataFrame(results)
pd.set_option('display.max_colwidth', None)
display(HTML(df.to_html(escape=False)))


> **BLEU Score -** <br>
> **Prompt -** Translate the following English legal text into Marathi exactly preserving all legal terms and clause structures. \n Use precise legal terminology. Do not simplify or reword any clauses. Maintain numbering, punctuation, and formatting as in the original text.<br>
**Model : Groq LLaMA 3.3-70B  Score : 60.44**

In [17]:
def gemini_translate(text: str) -> str:
    response = gemini_client.models.generate_content(
        model="gemini-2.0-flash",
        contents=f"""Translate the following English legal text into Marathi.
        ⚠️ STRICT REQUIREMENTS:
        - Translate word-for-word where possible.
        - Preserve all numbering, clause structure, indentation, and punctuation exactly.
        - Do not summarize or rephrase.
        - Maintain legal terminology used in Indian contracts.
        - Output only the translated text, no extra explanation.

        {text}""",
        config=types.GenerateContentConfig(
            temperature=0.0  # deterministic output
        )
    )
    return response.text

def gemini_translate_pro(text: str) -> str:
    response = gemini_client.models.generate_content(
        model="gemini-2.5-pro",   # 🔄 switched to 2.5 Pro
        contents=f"""Translate the following English legal text into Marathi.
        ⚠️ STRICT REQUIREMENTS:
        - Translate word-for-word where possible.
        - Preserve all numbering, clause structure, indentation, and punctuation exactly.
        - Do not summarize or rephrase.
        - Maintain legal terminology used in Indian contracts.
        - Output only the translated text, no extra explanation.

        {text}""",
        config=types.GenerateContentConfig(
            temperature=0.0  # deterministic output
        )     
    )
    return response.text

import re

def normalize_marathi_text(text: str) -> str:
    """
    Normalize Marathi legal text for BLEU scoring:
    - Lowercase
    - Remove extra spaces and newlines
    - Normalize punctuation
    - Normalize numbering (1. -> १., etc.)
    """
    # Lowercase
    text = text.lower()
    
    # Remove extra spaces, tabs, newlines
    text = re.sub(r"\s+", " ", text).strip()
    
    # Normalize punctuation
    text = text.replace("।", ".")           # Danda -> period
    text = text.replace("“", "\"").replace("”", "\"")
    text = text.replace("‘", "'").replace("’", "'")
    text = re.sub(r"[‐–—]", "-", text)       # Normalize dashes
    
    # Normalize numbering (optional, map English 1., 2., etc. to Marathi १., २.)
    def replace_numbering(match):
        eng_num = int(match.group(1))
        marathi_nums = "०१२३४५६७८९"
        marathi_num = "".join([marathi_nums[int(d)] for d in str(eng_num)])
        return f"{marathi_num}."
    
    text = re.sub(r"\b(\d+)\.", replace_numbering, text)

    return text
    
results = []

for prompt in prompts:  # ✅ only loop through prompts
    # --- Gemini ---
    gemini_translation = gemini_translate(f"{prompt}\n\n{english_text}")
    gemini_translation_pro = gemini_translate_pro(f"{prompt}\n\n{english_text}")

    # --- Normalize translations and reference ---
    gemini_norm = normalize_marathi_text(gemini_translation)
    gemini_pro_norm = normalize_marathi_text(gemini_translation_pro)
    reference_norm = normalize_marathi_text(reference_translation)

    # --- BLEU Scores ---
    gemini_bleu = sacrebleu.corpus_bleu([gemini_norm], [[reference_norm]]).score
    gemini_pro_bleu = sacrebleu.corpus_bleu([gemini_pro_norm], [[reference_norm]]).score
    # groq_bleu = sacrebleu.corpus_bleu([groq_translation], [[reference_translation]]).score
    # gpt120b_bleu = sacrebleu.corpus_bleu([gpt120b_translation], [[reference_translation]]).score

    results.append({
        "English Text": english_text[:100] + "...",
        "Prompt": prompt,
        "Google Gemini": gemini_translation,
        "Gemini BLEU": round(gemini_bleu, 2),
        "Google Gemini Pro": gemini_translation_pro,
        "Gemini Pro BLEU": round(gemini_pro_bleu, 2),
        # "Groq LLaMA 3.3-70B": groq_translation,
        # "Groq BLEU": round(groq_bleu, 2),
        # "GPT-OSS 120B": gpt120b_translation,
        # "GPT-OSS BLEU": round(gpt120b_bleu, 2),
        "Reference Translation": reference_translation[:100] + "..."
    })

df = pd.DataFrame(results)
pd.set_option('display.max_colwidth', None)
display(HTML(df.to_html(escape=False)))


,English Text,Prompt,Google Gemini,Gemini BLEU,Google Gemini Pro,Gemini Pro BLEU,Reference Translation
0,"\nSALE DEED\nThis Sale Deed is executed on this ___ day of ____, 20 at [Place], Maharashtra.\nBY AND BE...",Translate the following text into Marathi exactly preserving legal terms.,"विक्री करारनामा\nहा विक्री करारनामा आज दिनांक ___ ____, 20 रोजी [ठिकाण], महाराष्ट्र येथे निष्पादित केला आहे.\nयाद्वारे आणि यांच्यामध्ये\n1. [विक्रेत्याचे नाव], वय अंदाजे ___ वर्षे, रा. [पत्ता], यापुढे ""विक्रेता"" म्हणून संदर्भित (ज्यामध्ये, संदर्भास किंवा अर्थास विसंगत नसल्यास, त्यांचे वारस, कायदेशीर प्रतिनिधी, निष्पादक, प्रशासक आणि हक्क हस्तांतरिती यांचा समावेश असेल).\nआणि\n2. [खरेदीदाराचे नाव], वय अंदाजे ___ वर्षे, रा. [पत्ता], यापुढे ""खरेदीदार"" म्हणून संदर्भित (ज्यामध्ये, संदर्भास किंवा अर्थास विसंगत नसल्यास, त्यांचे वारस, कायदेशीर प्रतिनिधी, निष्पादक, प्रशासक आणि हक्क हस्तांतरिती यांचा समावेश असेल).\nविक्रेता आणि खरेदीदार यांना एकत्रितपणे ""पक्षकार"" म्हणून संदर्भित केले जाईल.\n________________________________________\nWHEREAS:\n1. विक्रेता भूखंडाचे संपूर्ण आणि कायदेशीर मालक आहे, ज्या भूखंडाचा सर्व्हे नंबर [____] आहे, आणि त्याचे क्षेत्रफळ [____] चौ.मी./चौ.फूट आहे, जो [गाव/शहर, तालुका, जिल्हा, महाराष्ट्र] येथे स्थित आहे, यापुढे ज्याचा उल्लेख ""सदर मालमत्ता"" असा केला जाईल.\n2. विक्रेत्याने सदर मालमत्ता खरेदीदाराला एकूण ₹[रक्कम] (रुपये [अक्षरात रक्कम]) च्या मोबदल्यात विकण्यास सहमती दर्शविली आहे.\n3. खरेदीदाराने सदर मालमत्ता विक्रेत्याकडून उपरोक्त मोबदल्यात, कोणत्याही भार, धारणाधिकार, शुल्क, गहाणखत किंवा खटलाबाजीपासून मुक्त खरेदी करण्यास सहमती दर्शविली आहे.\n________________________________________\nआता हा करारनामा खालीलप्रमाणे साक्षी देतो:\n1. विक्री आणि हस्तांतरण:\nविक्रेता, वर नमूद केलेल्या मोबदल्यासाठी, याद्वारे सदर मालमत्ता खरेदीदाराला पूर्णपणे विकतो, हस्तांतरित करतो आणि प्रदान करतो, त्यासोबत सर्व अधिकार, सुखाधिकार आणि त्यास संलग्न असलेल्या गोष्टीसुद्धा देतो.\n2. मोबदला:\n₹[रक्कम] चा एकूण विक्री मोबदला [भरणा करण्याची पद्धत: चेक/रोख/बँक हस्तांतरण] द्वारे विक्रेत्याला खरेदीदाराकडून प्राप्त झाला आहे, ज्याची पावती विक्रेता याद्वारे कबूल करतो.\n3. ताबा:\nविक्रेता याद्वारे सदर मालमत्तेचा प्रत्यक्ष आणि कायदेशीर ताबा या कराराच्या निष्पादनावर खरेदीदाराला देतो.\n4. मालकी हक्क आणि हमी:\nविक्रेता खात्री देतो की सदर मालमत्ता सर्व भार, कायदेशीर विवाद, दावे किंवा गहाणखत यांपासून मुक्त आहे. विक्रेता पुढे सहमत आहे की सदर मालमत्तेच्या मालकी हक्कामध्ये काही दोष आढळल्यास, त्यातून उद्भवणाऱ्या कोणत्याही नुकसानीस खरेदीदारास भरपाई देईल.\n5. खर्च आणि नोंदणी:\nस्टॅम्प ड्यूटी, नोंदणी शुल्क आणि इतर आनुषंगिक खर्चाशी संबंधित सर्व खर्च [खरेदार/विक्रेता यांच्यात परस्पर मान्यतेनुसार] द्वारे वहन केले जातील.\n6. नियामक कायदा:\nया विक्री कराराचे नियंत्रण आणि अर्थ लावणे भारतीय कायद्यानुसार केले जाईल, आणि विशेषतः, महाराष्ट्र स्टॅम्प ॲक्ट आणि मालमत्ता हस्तांतरण कायदा, १८८२ नुसार केले जाईल.\n________________________________________\nयाची साक्ष म्हणून, विक्रेता आणि खरेदीदार यांनी हा विक्री करारनामा वर नमूद केलेल्या दिवशी, महिन्यात आणि वर्षी निष्पादित केला आहे.\nसही, शिक्का व सुपूर्द:\nविक्रेता:\n(सही) ___________________\nनाव: _______________________\nपत्ता: ____________________\nखरेदीदार:\n(सही) ___________________\nनाव: _______________________\nपत्ता: ____________________\n witnesses:\n1. नाव: ___________________ सही: _______________ पत्ता: _______________\n2. नाव: ___________________ सही: _______________ पत्ता: _______________\n",66.57,"विक्रीपत्र\nहे विक्रीपत्र आज या ___ दिवशी ____, २० रोजी [ठिकाण], महाराष्ट्र येथे निष्पादित करण्यात आले आहे.\nयांच्याद्वारे आणि यांच्यामध्ये\n१. [विक्रेत्याचे नाव], वय अंदाजे ___ वर्षे, राहणार [पत्ता], यापुढे ""विक्रेता"" म्हणून संबोधले जाईल (ज्या अभिव्यक्तीमध्ये, जोपर्यंत संदर्भ किंवा अर्थाच्या विरूद्ध नसेल तोपर्यंत, त्यांचे/त्यांचे वारस, कायदेशीर प्रतिनिधी, मृत्युपत्र व्यवस्थापक, प्रशासक आणि अभिहस्तांकिती यांचा समावेश असेल).\nआणि\n२. [खरेदीदाराचे नाव], वय अंदाजे ___ वर्षे, राहणार [पत्ता], यापुढे ""खरेदीदार"" म्हणून संबोधले जाईल (ज्या अभिव्यक्तीमध्ये, जोपर्यंत संदर्भ किंवा अर्थाच्या विरूद्ध नसेल तोपर्यंत, त्यांचे/त्यांचे वारस, कायदेशीर प्र

>**Bleu Score of models:**<br>
Prompt : <br>
"Translate the following English legal text into Marathi. Ensure all key legal terms like 'possession', 'Seller', 'Buyer', and 'government dues' \n are correctly translated. Maintain full context and structure. If a term has multiple possible translations, choose the one used in Indian legal documents."<br>
**gemini-2.0-flash score**: 68.40 <br>
**gemini-2.5-pro : 69.03** <br><br>
Prompt : <br>
"Translate the following English legal text into Marathi as a concise but legally accurate summary. Keep all rights, obligations, and \n clauses intact, but reduce redundancy. Suitable for executives or quick review."<br>
**gemini-2.0-flash score**: 68.71 <br>
**gemini-2.5-pro** : 68.27 <br><br>
Prompt : <br>
"Translate the following English legal text into Marathi as a concise but legally accurate summary. Keep all rights, obligations, and \n clauses intact, but reduce redundancy. Suitable for executives or quick review."<br>
**gemini-2.0-flash score**: 68.71 <br>
**gemini-2.5-pro** : 68.27 <br><br>
Prompt : <br>
"Translate the following English legal text into Marathi exactly preserving all legal terms and clause structures. \n Use precise legal terminology. Do not simplify or reword any clauses. Maintain numbering, punctuation, and formatting as in the original text."<br>
**gemini-2.0-flash score**: 67.07 <br>
**gemini-2.5-pro** : 67.51 <br><br>
Prompt : <br>
"Translate the following text into Marathi keeping professional/legal tone."<br>
**gemini-2.0-flash score**: 67.90 <br>
**gemini-2.5-pro** : 65.38 <br><br>
Prompt : <br>
"Translate the following text into Marathi in simple, easy-to-understand Marathi."<br>
**gemini-2.0-flash score**: 62.38 <br>
**gemini-2.5-pro** : 67.02 <br><br>
Prompt : <br>
"Translate the following text into Marathi exactly preserving legal terms."<br>
**gemini-2.0-flash score**: 66.57 <br>
**gemini-2.5-pro** : 66.43 <br>

In [4]:
def gemini_translate(text: str) -> str:
    response = gemini_client.models.generate_content(
        model="gemini-2.0-flash",
        contents=f"""
        Translate the following English legal text into Marathi in a formal legal register. 
        Use terminology consistent with Indian contracts and deeds. 
        Do not simplify or modernize the language. 
        Preserve original clause numbering, structure, and formatting. 
        Output only the translated document, nothing else.


        {text}""",
        config=types.GenerateContentConfig(
            temperature=0.0  # deterministic output
        )
    )
    return response.text

def gemini_translate_pro(text: str) -> str:
    response = gemini_client.models.generate_content(
        model="gemini-2.5-pro",   # 🔄 switched to 2.5 Pro
        contents=f"""
        Translate the following English legal text into Marathi in a formal legal register. 
        Use terminology consistent with Indian contracts and deeds. 
        Do not simplify or modernize the language. 
        Preserve original clause numbering, structure, and formatting. 
        Output only the translated document, nothing else.


        {text}""",
        config=types.GenerateContentConfig(
            temperature=0.0  # deterministic output
        )     
    )
    return response.text


# ---------------- Run Translations ----------------
import re

def normalize_marathi_text(text: str) -> str:
    """
    Normalize Marathi legal text for BLEU scoring:
    - Lowercase
    - Remove extra spaces and newlines
    - Normalize punctuation
    - Normalize numbering (1. -> १., etc.)
    """
    # Lowercase
    text = text.lower()
    
    # Remove extra spaces, tabs, newlines
    text = re.sub(r"\s+", " ", text).strip()
    
    # Normalize punctuation
    text = text.replace("।", ".")           # Danda -> period
    text = text.replace("“", "\"").replace("”", "\"")
    text = text.replace("‘", "'").replace("’", "'")
    text = re.sub(r"[‐–—]", "-", text)       # Normalize dashes
    
    # Normalize numbering (optional, map English 1., 2., etc. to Marathi १., २.)
    def replace_numbering(match):
        eng_num = int(match.group(1))
        marathi_nums = "०१२३४५६७८९"
        marathi_num = "".join([marathi_nums[int(d)] for d in str(eng_num)])
        return f"{marathi_num}."
    
    text = re.sub(r"\b(\d+)\.", replace_numbering, text)

    return text

results = []

for prompt in prompts:  # ✅ only loop through prompts
    # --- Gemini ---
    gemini_translation = gemini_translate(f"{prompt}\n\n{english_text}")
    gemini_translation_pro = gemini_translate_pro(f"{prompt}\n\n{english_text}")

    # --- Normalize translations and reference ---
    gemini_norm = normalize_marathi_text(gemini_translation)
    gemini_pro_norm = normalize_marathi_text(gemini_translation_pro)
    reference_norm = normalize_marathi_text(reference_translation)

    # --- BLEU Scores ---
    gemini_bleu = sacrebleu.corpus_bleu([gemini_norm], [[reference_norm]]).score
    gemini_pro_bleu = sacrebleu.corpus_bleu([gemini_pro_norm], [[reference_norm]]).score

    results.append({
        "English Text": english_text[:100] + "...",
        "Prompt": prompt,
        "Google Gemini": gemini_translation,
        "Gemini BLEU": round(gemini_bleu, 2),
        "Google Gemini Pro": gemini_translation_pro,
        "Gemini Pro BLEU": round(gemini_pro_bleu, 2),
        "Reference Translation": reference_translation[:100] + "..."
    })

df = pd.DataFrame(results)
pd.set_option('display.max_colwidth', None)
display(HTML(df.to_html(escape=False)))


,English Text,Prompt,Google Gemini,Gemini BLEU,Google Gemini Pro,Gemini Pro BLEU,Reference Translation
0,"\nSALE DEED\nThis Sale Deed is executed on this ___ day of ____, 20 at [Place], Maharashtra.\nBY AND BE...",Translate the following text into Marathi exactly preserving legal terms.,"विक्रयपत्र\nहे विक्रयपत्र आज दिनांक ___ रोजी ____, इ.स. २०__ [ठिकाण], महाराष्ट्र येथे निष्पादित केले आहे.\nयाद्वारे आणि यांच्यामध्ये:\n१. [विक्रेत्याचे नाव], वय अंदाजे ___ वर्षे, रा. [पत्ता], यापुढे “विक्रेता” म्हणून संदर्भित (ज्यामध्ये, संदर्भास किंवा अर्थास विसंगत नसल्यास, त्यांचे वारस, कायदेशीर प्रतिनिधी, निष्पादक, प्रशासक आणि हक्क हस्तांतरिती यांचा समावेश असेल).\nआणि\n२. [खरेदीदाराचे नाव], वय अंदाजे ___ वर्षे, रा. [पत्ता], यापुढे “खरेदीदार” म्हणून संदर्भित (ज्यामध्ये, संदर्भास किंवा अर्थास विसंगत नसल्यास, त्यांचे वारस, कायदेशीर प्रतिनिधी, निष्पादक, प्रशासक आणि हक्क हस्तांतरिती यांचा समावेश असेल).\nविक्रेता आणि खरेदीदार यांना एकत्रितपणे “पक्षकार” म्हणून संबोधले जाईल.\n________________________________________\nWHEREAS (म्हणून):\n१. विक्रेता भूखंडाचे संपूर्ण आणि कायदेशीर मालक आहेत, ज्या भूखंडाचा सर्व्हे नंबर [____] आहे, आणि त्याचे क्षेत्रफळ [____] चौ.मी./चौ.फूट आहे, जो [गाव/शहर, तालुका, जिल्हा, महाराष्ट्र] येथे स्थित आहे, यापुढे त्यास ""सदर मालमत्ता"" म्हणून संबोधले जाईल.\n२. विक्रेत्याने सदर मालमत्ता खरेदीदारास एकूण ₹[रक्कम] (रुपये [अक्षरी रक्कम]) च्या मोबदल्यात विक्री करण्याचे मान्य केले आहे.\n३. खरेदीदाराने सदर मालमत्ता विक्रेत्याकडून उपरोक्त मोबदल्यात, कोणत्याही भार, धारणाधिकार, दायित्व, गहाण किंवा खटलाबाजी विरहित खरेदी करण्याचे मान्य केले आहे.\n________________________________________\nआता हे দলিল साक्ष देते की:\n१. विक्री आणि हस्तांतरण:\nविक्रेता, उपरोक्त नमूद केलेल्या मोबदल्याच्या बदल्यात, सदर मालमत्ता खरेदीदारास सर्व हक्क, सुखाधिकार आणि उपांगिकांसह पूर्णपणे विक्री, हस्तांतरण आणि प्रदान करीत आहेत.\n२. मोबदला:\nएकूण विक्री मोबदला ₹[रक्कम] विक्रेत्याने खरेदीदाराकडून [भरणा करण्याची पद्धत: धनादेश/रोख/बँक हस्तांतरण] द्वारे प्राप्त केला आहे, ज्याची पावती विक्रेता याद्वारे देत आहे.\n३. ताबे:\nविक्रेता याद्वारे सदर मालमत्तेचा प्रत्यक्ष आणि कायदेशीर ताबा या दस्ताच्या निष्पादनावर खरेदीदारास देत आहे.\n४. हक्क व हमी:\nविक्रेता हमी देतात की सदर मालमत्ता सर्व भार, कायदेशीर विवाद, दावे किंवा गहाण यांपासून मुक्त आहे. विक्रेता पुढे खरेदीदारास सदर मालमत्तेच्या हक्कामध्ये कोणत्याही दोषांमुळे उद्भवणाऱ्या कोणत्याही नुकसानी भरपाई देण्यास सहमत आहे.\n५. खर्च आणि नोंदणी:\n मुद्रांक शुल्क, नोंदणी शुल्क आणि इतर आनुषंगिक खर्चाशी संबंधित सर्व खर्च [खरेदीदार/विक्रेता यांच्यात परस्पर मान्यतेनुसार] द्वारे वहन केले जातील.\n६. शासित कायदा:\nहे विक्रयपत्र भारताच्या कायद्यानुसार आणि विशेषतः महाराष्ट्र मुद्रांक अधिनियम आणि मालमत्ता हस्तांतरण अधिनियम, १८८२ नुसार शासित आणि अन्वयार्थ लावले जाईल.\n________________________________________\nयाची साक्ष म्हणून, विक्रेता आणि खरेदीदार यांनी हे विक्रयपत्र वर नमूद केलेल्या दिवशी, महिन्यात आणि वर्षी निष्पादित केले आहे.\nसही, शिक्का व सुपूर्द:\nविक्रेता:\n(सही) ___________________\nनाव: _______________________\nपत्ता: ____________________\nखरेदीदार:\n(सही) ___________________\nनाव: _______________________\nपत्ता: ____________________\n साक्षीदार:\n१. नाव: ___________________ सही: _______________ पत्ता: _______________\n२. नाव: ___________________ सही: _______________ पत्ता: _______________\n",65.33,"विक्रीखत\nहे विक्रीखत आज दिनांक ___ रोजी, महिना ____, सन २०__ रोजी, [ठिकाण], महाराष्ट्र येथे निष्पादित करण्यात आले आहे.\nयांच्या दरम्यान\n१. [विक्रेत्याचे नाव], वय वर्षे अंदाजे ___, राहणार [पत्ता], यांना यापुढे 'विक्रेता' असे संबोधण्यात येईल (या अभिव्यक्तीमध्ये, जोपर्यंत संदर्भ किंवा अर्थाच्या विरुद्ध नसेल, त्यांचे वारस, कायदेशीर प्रतिनिधी, निष्पादक, प्रशासक आणि अभिहस्तांकिती यांचा समावेश असेल).\nआणि\n२. [खरेदीदाराचे नाव], वय वर्षे अंदाजे ___, राहणार [पत्ता], यांना यापुढे 'खरेदीदार' असे संबोधण्यात येईल (या अभिव्यक्तीमध्ये, जोपर्यंत संदर्भ किंवा अर्थाच्या विरुद्ध नसेल, त्यांचे वारस, कायदेशीर प्रतिनिधी, निष्पादक, प्रशासक आणि अभिहस्तांकिती यांचा समावेश असेल).\nविक्रेता आणि खरेदीदार यांना एकत्रितपणे 'पक्षकार' असे संबोधण्यात येईल.\n_____________

>**Bleu Score of each model:**<br>
**Prompt :** <br>
"Translate the following text into Marathi in simple, easy-to-understand Marathi."<br>
**gemini-2.0-flash : 69.03**<br>
**gemini-2.5-pro : 63.32** <br><br>
**Prompt :** <br>
"Translate the following English legal text into Marathi as a concise but legally accurate summary. Keep all rights, obligations, and \n clauses intact, but reduce redundancy. Suitable for executives or quick review."<br>
**gemini-2.0-flash : 68.29**<br>
**gemini-2.5-pro : 64.02** <br><br>
**Prompt :** <br>
"Translate the following text into Marathi exactly preserving legal terms."<br>
**gemini-2.0-flash : 65.33**<br>
**gemini-2.5-pro : 66.26** <br><br>
**Prompt :** <br>
"Translate the following English legal text into Marathi exactly preserving all legal terms and clause structures. \n Use precise legal terminology. Do not simplify or reword any clauses. Maintain numbering, punctuation, and formatting as in the original text."<br>
**gemini-2.0-flash : 66.67**<br>
**gemini-2.5-pro : 65.28** <br><br>
**Prompt :** <br>
"Translate the following English legal text into Marathi. Ensure all key legal terms like 'possession', 'Seller', 'Buyer', and 'government dues' \n are correctly translated. Maintain full context and structure. If a term has multiple possible translations, choose the one used in Indian legal documents."<br>
**gemini-2.0-flash : 64.66**<br>
**gemini-2.5-pro : 64.82** <br><br>
**Prompt :** <br>
"Translate the following text into Marathi keeping professional/legal tone."<br>
**gemini-2.0-flash : 63.77**<br>
**gemini-2.5-pro : 63.41** <br><br>

In [22]:
# ---------- Translation Functions (Only System Prompt) ----------
def groq_translate(text: str) -> str:
    return client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=[
            {"role": "system", "content": """
            You are a professional legal translator specializing in Indian contracts, deeds, and official agreements. 

Task: Translate the given English text into Marathi while adhering to the following rules:

1. Preserve all legal terminology exactly. Use standard Marathi legal terms used in Indian courts and official documents. 
2. Maintain the original clause numbering, bullet points, headings, and formatting. Do not alter the structure of the document.
3. Keep sentences formal, precise, and legally accurate. Do not simplify, modernize, or paraphrase.
4. Output only the translated document. Do not include explanations, summaries, or any additional text.
5. Ensure the translation is deterministic: repeated translations of the same text must yield the same result.

Translate the following English text into Marathi:

    """},
            {"role": "user", "content": text}
        ],
        temperature=0.0,
        max_tokens=1500,
    ).choices[0].message.content

def gpt_120b_translate(text: str) -> str:
    return client.chat.completions.create(
        model="openai/gpt-oss-120b",
        messages=[
            {"role": "system", "content": """
            You are a professional legal translator specializing in Indian contracts, deeds, and official agreements. 

Task: Translate the given English text into Marathi while adhering to the following rules:

1. Preserve all legal terminology exactly. Use standard Marathi legal terms used in Indian courts and official documents. 
2. Maintain the original clause numbering, bullet points, headings, and formatting. Do not alter the structure of the document.
3. Keep sentences formal, precise, and legally accurate. Do not simplify, modernize, or paraphrase.
4. Output only the translated document. Do not include explanations, summaries, or any additional text.
5. Ensure the translation is deterministic: repeated translations of the same text must yield the same result.

Translate the following English text into Marathi:

    """},
            {"role": "user", "content": text}
        ],
        temperature=0.0,
        max_tokens=1500,
    ).choices[0].message.content


import re

def normalize_marathi_text(text: str) -> str:
    """
    Normalize Marathi legal text for BLEU scoring:
    - Lowercase
    - Remove extra spaces and newlines
    - Normalize punctuation
    - Normalize numbering (1. -> १., etc.)
    """
    # Lowercase
    text = text.lower()
    
    # Remove extra spaces, tabs, newlines
    text = re.sub(r"\s+", " ", text).strip()
    
    # Normalize punctuation
    text = text.replace("।", ".")           # Danda -> period
    text = text.replace("“", "\"").replace("”", "\"")
    text = text.replace("‘", "'").replace("’", "'")
    text = re.sub(r"[‐–—]", "-", text)       # Normalize dashes
    
    # Normalize numbering (optional, map English 1., 2., etc. to Marathi १., २.)
    def replace_numbering(match):
        eng_num = int(match.group(1))
        marathi_nums = "०१२३४५६७८९"
        marathi_num = "".join([marathi_nums[int(d)] for d in str(eng_num)])
        return f"{marathi_num}."
    
    text = re.sub(r"\b(\d+)\.", replace_numbering, text)

    return text
# ---------------- Run Translations ----------------
results = []

for prompt in prompts:
    groq_translation = groq_translate(f"{prompt}\n\n{english_text}")
    gpt120b_translation = gpt_120b_translate(f"{prompt}\n\n{english_text}")

    groq_norm = normalize_marathi_text(groq_translation)
    gpt120b_norm = normalize_marathi_text(gpt120b_translation)
    reference_norm = normalize_marathi_text(reference_translation)

    groq_bleu = sacrebleu.corpus_bleu([groq_norm], [[reference_norm]]).score
    gpt120b_bleu = sacrebleu.corpus_bleu([gpt120b_norm], [[reference_norm]]).score

    results.append({
        "English Text": english_text[:100] + "...",
        "Prompt": prompt,
        "Groq LLaMA 3.3-70B": groq_translation,
        "Groq BLEU": round(groq_bleu, 2),
        "GPT-OSS 120B": gpt120b_translation,
        "GPT-OSS BLEU": round(gpt120b_bleu, 2),
        "Reference Translation": reference_translation[:100] + "..."
    })


df = pd.DataFrame(results)
pd.set_option('display.max_colwidth', None)
display(HTML(df.to_html(escape=False)))

,English Text,Prompt,Groq LLaMA 3.3-70B,Groq BLEU,GPT-OSS 120B,GPT-OSS BLEU,Reference Translation
0,"\nSALE DEED\nThis Sale Deed is executed on this ___ day of ____, 20 at [Place], Maharashtra.\nBY AND BE...",Translate the following text into Marathi exactly preserving legal terms.,"विक्री पत्र\nहे विक्री पत्र दिनांक ___ ____, 20 येथे [स्थान], महाराष्ट्र येथे केले आहे.\nभावी द्वारे\n1. [विक्रेत्याचे नाव], वय सुमारे ___ वर्षे, सध्या राहिवाश [पत्ता] येथे, यापुढे ""विक्रेता"" (या शब्दाचा अर्थ अशा प्रकारे घेतला जाईल की ज्यामध्ये त्याचे/तिचे वारस, कायदेशीर प्रतिनिधी, कार्यकारी, प्रशासक आणि वारस यांचा समावेश असेल) म्हणून संबोधले जाईल.\nआणि\n2. [गिरवणाराचे नाव], वय सुमारे ___ वर्षे, सध्या राहिवाश [पत्ता] येथे, यापुढे ""गिरवणारा"" (या शब्दाचा अर्थ अशा प्रकारे घेतला जाईल की ज्यामध्ये त्याचे/तिचे वारस, कायदेशीर प्रतिनिधी, कार्यकारी, प्रशासक आणि वारस यांचा समावेश असेल) म्हणून संबोधले जाईल.\nविक्रेता आणि गिरवणारा एकत्रितपणे ""पक्ष"" म्हणून संबोधले जातील.\n________________________________________\nयाविषयी:\n1. विक्रेता हा सर्वेसर्वा आणि कायदेशीर मालक आहे अशा जागेचा, सर्वे नंबर [____], क्षेत्रफळ [____] चौरस मी. /चौरस फुटांचा, [गाव/शहर, तालुका, जिल्हा, महाराष्ट्र] येथे स्थित, यापुढे ""सांगितलेली जागा"" म्हणून संबोधले जाईल.\n2. विक्रेत्याने सांगितलेल्या जागेचा विक्री गिरवणाराला एकूण ₹[रकम] (रुपये [शब्दातील रकम]) या मोबदल्यात करण्यास सहमती दर्शविली आहे.\n3. गिरवणाराने विक्रेत्याकडून सांगितलेली जागा मुक्त करण्याच्या बंधने, लायणे, आकार, हप्ते, किंवा खटला यांच्याशिवाय मोबदल्यासाठी घेण्यास सहमती दर्शविली आहे.\n________________________________________\nआता ही पत्र अशी साक्ष देते:\n1. विक्री आणि हस्तांतरण:\nविक्रेत्याने वरील नमूद केलेल्या मोबदल्यासाठी सांगितलेली जागा गिरवणाराला पूर्णपणे विक्री केली, हस्तांतरित केले आणि सोपविले, तिला संबंधित सर्व अधिकार, सुविधा आणि संलग्नकांसह.\n2. मोबदला:\nएकूण विक्री मोबदल्याची ₹[रकम] रक्कम विक्रेत्याकडे गिरवणाराकडून [चेक/नगद/बँक हस्तांतरण] पद्धतीने मिळाली आहे, ज्याची पावती विक्रेत्याने दिली आहे.\n3. ताबा:\nविक्रेत्याने सांगितलेली जागा या पत्रावर स्वाक्षरी झाल्यानंतर गिरवणाराला भौतिक आणि कायदेशीर ताबा दिला आहे.\n4. मालमत्ता आणि हमी:\nविक्रेत्याने अशी खात्री दिली आहे की सांगितलेली जागा कोणत्याही बंधने, कायदेशीर विवाद, दावे किंवा हप्त्यांपासून मुक्त आहे. विक्रेत्याने आणखी असे म्हटले आहे की सांगितलेल्या जागेच्या मालकीहक्कातील कोणत्याही दोषामुळे गिरवणाराला होणारी कोणतीही नुकसान भरपाई देण्यास तयार आहे.\n5. खर्च आणि नोंदणी:\nस्ताम्प शुल्क, नोंदणी शुल्क आणि इतर अप्रत्यक्ष खर्च यांचा संबंध असलेले सर्व खर्च [गिरवणारा/विक्रेता परस्पर सहमतीने] वहन करणार आहे.\n6. नियामक कायदा:\nही विक्री पत्र भारतीय कायद्यांनुसार आणि विशेषतः महाराष्ट्र स्टॅम्प अधिनियम आणि संपत्ती हस्तांतरण अधिनियम, 1882 अनुसार नियंत्रित केली जाईल.\n________________________________________\nयाची साक्ष आहे की विक्रेता आणि गिरवणाराने वर नमूद केलेल्या दिनांक, महिना आणि वर्षी ही विक्री पत्र केली आहे.\nस्वाक्षरी, मोहोर आणि वितरण:\nविक्रेता:\n(स्वाक्षरी) ___________________\nनाव: _______________________\nस्थावर पत्ता: ____________________\nगिरवणारा:\n(स्वाक्षरी) ___________________\nनाव: _______________________\nस्थावर पत्ता: ____________________\nसाक्षीदार:\n1. नाव: ___________________ स्वाक्षरी: _______________ स्थावर पत्ता:",56.12,"SALE DEED \nहे विक्री करारपत्र ____ दिनांक ____, 20 रोजी [Place], महाराष्ट्र येथे अंमलात आणले गेले आहे. \n\nBY AND BETWEEN \n\n1. [Seller Name], वय सुमारे ___ वर्षे, राहते [Address], पुढे “विक्रेता” म्हणून उल्लेखित (ज्याचा अर्थ, संदर्भ किंवा अर्थाच्या विरोधात नसल्यास, त्याचे वारस, कायदेशीर प्रतिनिधी, कार्यकारी, प्रशासक व हस्तांतरित व्यक्ती यांचा समावेश होईल). \nAND \n2. [Buyer Name], वय सुमारे ___ वर्षे, राहते [Address], पुढे “खरेदीदार” म्हणून उल्लेखित (ज्याचा अर्थ, संदर्भ किंवा अर्थाच्या विरोधात नसल्यास, त्याचे वारस, कायदेशीर प्रतिनिधी, कार्यकारी, प्रशासक व हस्तांतरित व्यक्ती यांचा समावेश होईल). \n\nविक्रेता व खरेदीदार यांना एकत्रितपणे “पक्ष” म्हणून उल्लेखित केले आहे. \n\n________________________________________ \n\nWHEREAS: \n\n1. विक्रेत्याचा सर्वसाधारण व कायदेशीर मालकीचा भूखंड, सर्वे क्रमांक [____], क्षेत्रफळ [____] चौरस मी./चौरस फू

> **BLEU Score -** <br>
**Prompt -**
Translate the following text into Marathi in simple, easy-to-understand Marathi.<br>
**Groq LLaMA 3.3-70B :** 65.62<br>
**GPT-OSS 120B :** 20.67<br><br>
**Prompt -**
Translate the following text into Marathi keeping professional/legal tone.<br>
**Groq LLaMA 3.3-70B :** 63.88<br>
**GPT-OSS 120B :** 22.40<br><br>
**Prompt -**
Translate the following text into Marathi exactly preserving legal terms.<br>
**Groq LLaMA 3.3-70B :** 56.12<br>
**GPT-OSS 120B :** 14.56<br><br>
**Prompt -**
Translate the following English legal text into Marathi as a concise but legally accurate summary. Keep all rights, obligations, and \n clauses intact, but reduce redundancy. Suitable for executives or quick review.<br>
**Groq LLaMA 3.3-70B :** 55.57<br>
**GPT-OSS 120B :** 27.68<br><br>
**Prompt -**
Translate the following English legal text into Marathi. Ensure all key legal terms like 'possession', 'Seller', 'Buyer', and 'government dues' \n are correctly translated. Maintain full context and structure. If a term has multiple possible translations, choose the one used in Indian legal documents.<br>
**Groq LLaMA 3.3-70B :** 54.07<br>
**GPT-OSS 120B :** 23.00<br><br>
**Prompt -**
Translate the following English legal text into Marathi exactly preserving all legal terms and clause structures. \n Use precise legal terminology. Do not simplify or reword any clauses. Maintain numbering, punctuation, and formatting as in the original text.<br>
**Groq LLaMA 3.3-70B :** 39.03<br>
**GPT-OSS 120B :** 0.0



In [7]:
def gemini_translate(text: str) -> str:
    response = gemini_client.models.generate_content(
        model="gemini-2.0-flash",
        contents=f"""
        Translate the following English legal text into Marathi in a formal legal register. 
        Use terminology consistent with Indian contracts and deeds. 
        Do not simplify or modernize the language. 
        Preserve original clause numbering, structure, and formatting. 
        Output only the translated document, nothing else.


        {text}""",
        config=types.GenerateContentConfig(
            temperature=0.0  # deterministic output
        )
    )
    return response.text

def gemini_translate_pro(text: str) -> str:
    response = gemini_client.models.generate_content(
        model="gemini-2.5-pro",   # 🔄 switched to 2.5 Pro
        contents=f"""
        Translate the following English legal text into Marathi in a formal legal register. 
        Use terminology consistent with Indian contracts and deeds. 
        Do not simplify or modernize the language. 
        Preserve original clause numbering, structure, and formatting. 
        Output only the translated document, nothing else.


        {text}""",
        config=types.GenerateContentConfig(
            temperature=0.0  # deterministic output
        )     
    )
    return response.text


# ---------------- Run Translations ----------------
import re

def normalize_marathi_text(text: str) -> str:
    """
    Normalize Marathi legal text for BLEU scoring:
    - Lowercase
    - Remove extra spaces and newlines
    - Normalize punctuation
    - Normalize numbering (1. -> १., etc.)
    """
    # Lowercase
    text = text.lower()
    
    # Remove extra spaces, tabs, newlines
    text = re.sub(r"\s+", " ", text).strip()
    
    # Normalize punctuation
    text = text.replace("।", ".")           # Danda -> period
    text = text.replace("“", "\"").replace("”", "\"")
    text = text.replace("‘", "'").replace("’", "'")
    text = re.sub(r"[‐–—]", "-", text)       # Normalize dashes
    
    # Normalize numbering (optional, map English 1., 2., etc. to Marathi १., २.)
    def replace_numbering(match):
        eng_num = int(match.group(1))
        marathi_nums = "०१२३४५६७८९"
        marathi_num = "".join([marathi_nums[int(d)] for d in str(eng_num)])
        return f"{marathi_num}."
    
    text = re.sub(r"\b(\d+)\.", replace_numbering, text)

    return text

results = []

for prompt in prompts:  # ✅ only loop through prompts
    # --- Gemini ---
    gemini_translation = gemini_translate(f"{prompt}\n\n{english_text}")
    gemini_translation_pro = gemini_translate_pro(f"{prompt}\n\n{english_text}")

    # --- Normalize translations and reference ---
    gemini_norm = normalize_marathi_text(gemini_translation)
    gemini_pro_norm = normalize_marathi_text(gemini_translation_pro)
    reference_norm = normalize_marathi_text(reference_translation)

    # --- BLEU Scores ---
    ref_tokens = reference_norm.split()

    gemini_bleu = sentence_bleu(
        [ref_tokens],
        gemini_norm.split(),
        weights=(0.25, 0.25, 0.25, 0.25)
    )

    gemini_pro_bleu = sentence_bleu(
        [ref_tokens],
        gemini_pro_norm.split(),
        weights=(0.25, 0.25, 0.25, 0.25)
    )


    results.append({
        "English Text": english_text[:100] + "...",
        "Prompt": prompt,
        "Google Gemini": gemini_translation,
        "Gemini BLEU": round(gemini_bleu, 2),
        "Google Gemini Pro": gemini_translation_pro,
        "Gemini Pro BLEU": round(gemini_pro_bleu, 2),
        "Reference Translation": reference_translation[:100] + "..."
    })

df = pd.DataFrame(results)
pd.set_option('display.max_colwidth', None)
display(HTML(df.to_html(escape=False)))


,English Text,Prompt,Google Gemini,Gemini BLEU,Google Gemini Pro,Gemini Pro BLEU,Reference Translation
0,"\nSALE DEED\nThis Sale Deed is executed on this ___ day of ____, 20 at [Place], Maharashtra.\nBY AND BE...",Translate the following text into Marathi exactly preserving legal terms.,"विक्रयपत्र\nहे विक्रयपत्र आज दिनांक ___ रोजी ____, इ.स. २०__ [ठिकाण], महाराष्ट्र येथे निष्पादित केले आहे.\nयाद्वारे आणि यांच्यामध्ये:\n१. [विक्रेत्याचे नाव], वय अंदाजे ___ वर्षे, रा. [पत्ता], यापुढे “विक्रेता” म्हणून संदर्भित (ज्यामध्ये, संदर्भास किंवा अर्थास विसंगत नसल्यास, त्यांचे वारस, कायदेशीर प्रतिनिधी, निष्पादक, प्रशासक आणि हक्क हस्तांतरिती यांचा समावेश असेल).\nआणि\n२. [खरेदीदाराचे नाव], वय अंदाजे ___ वर्षे, रा. [पत्ता], यापुढे “खरेदीदार” म्हणून संदर्भित (ज्यामध्ये, संदर्भास किंवा अर्थास विसंगत नसल्यास, त्यांचे वारस, कायदेशीर प्रतिनिधी, निष्पादक, प्रशासक आणि हक्क हस्तांतरिती यांचा समावेश असेल).\nविक्रेता आणि खरेदीदार यांना एकत्रितपणे “पक्षकार” म्हणून संबोधले जाईल.\n________________________________________\nWHEREAS (म्हणून):\n१. विक्रेता भूखंडाचे संपूर्ण आणि कायदेशीर मालक आहेत, ज्या भूखंडाचा सर्व्हे नंबर [____] आहे, आणि त्याचे क्षेत्रफळ [____] चौ.मी./चौ.फूट आहे, जो [गाव/शहर, तालुका, जिल्हा, महाराष्ट्र] येथे स्थित आहे, यापुढे त्यास ""सदर मालमत्ता"" म्हणून संबोधले जाईल.\n२. विक्रेत्याने सदर मालमत्ता खरेदीदारास एकूण ₹[रक्कम] (रुपये [अक्षरी रक्कम]) च्या मोबदल्यात विक्री करण्याचे मान्य केले आहे.\n३. खरेदीदाराने सदर मालमत्ता विक्रेत्याकडून उपरोक्त मोबदल्यात, कोणत्याही भार, धारणाधिकार, दायित्व, गहाण किंवा खटलाबाजी विरहित खरेदी करण्याचे मान्य केले आहे.\n________________________________________\nआता हे দলিল साक्ष देते की:\n१. विक्री आणि हस्तांतरण:\nविक्रेता, उपरोक्त नमूद केलेल्या मोबदल्याच्या बदल्यात, सदर मालमत्ता खरेदीदारास सर्व हक्क, सुखाधिकार आणि उपांगिकांसह पूर्णपणे विक्री, हस्तांतरण आणि प्रदान करीत आहेत.\n२. मोबदला:\nएकूण विक्री मोबदला ₹[रक्कम] विक्रेत्याने खरेदीदाराकडून [भरणा करण्याची पद्धत: धनादेश/रोख/बँक हस्तांतरण] द्वारे प्राप्त केला आहे, ज्याची पावती विक्रेता याद्वारे देत आहे.\n३. ताबे:\nविक्रेता याद्वारे सदर मालमत्तेचा प्रत्यक्ष आणि कायदेशीर ताबा या दस्ताच्या निष्पादनावर खरेदीदारास देत आहे.\n४. हक्क व हमी:\nविक्रेता हमी देतात की सदर मालमत्ता सर्व भार, कायदेशीर विवाद, दावे किंवा गहाण यांपासून मुक्त आहे. विक्रेता पुढे खरेदीदारास सदर मालमत्तेच्या हक्कामध्ये कोणत्याही दोषांमुळे उद्भवणाऱ्या कोणत्याही नुकसानी भरपाई देण्यास सहमत आहे.\n५. खर्च आणि नोंदणी:\n मुद्रांक शुल्क, नोंदणी शुल्क आणि इतर आनुषंगिक खर्चाशी संबंधित सर्व खर्च [खरेदीदार/विक्रेता यांच्यात परस्पर मान्यतेनुसार] द्वारे वहन केले जातील.\n६. शासित कायदा:\nहे विक्रयपत्र भारताच्या कायद्यानुसार आणि विशेषतः महाराष्ट्र मुद्रांक अधिनियम आणि मालमत्ता हस्तांतरण अधिनियम, १८८२ नुसार शासित आणि अन्वयार्थ लावले जाईल.\n________________________________________\nयाची साक्ष म्हणून, विक्रेता आणि खरेदीदार यांनी हे विक्रयपत्र वर नमूद केलेल्या दिवशी, महिन्यात आणि वर्षी निष्पादित केले आहे.\nसही, शिक्का व सुपूर्द:\nविक्रेता:\n(सही) ___________________\nनाव: _______________________\nपत्ता: ____________________\nखरेदीदार:\n(सही) ___________________\nनाव: _______________________\nपत्ता: ____________________\n साक्षीदार:\n१. नाव: ___________________ सही: _______________ पत्ता: _______________\n२. नाव: ___________________ सही: _______________ पत्ता: _______________\n",0.24,"विक्रीखत\nहे विक्रीखत आज रोजी, दिनांक ___ महिना ____, सन २०__ रोजी, [ठिकाण], महाराष्ट्र येथे निष्पादित करण्यात आले आहे.\nयांमध्ये\n१. [विक्रेत्याचे नाव], वय वर्षे अंदाजे ___, राहणार [पत्ता], यापुढे 'विक्रेता' म्हणून संबोधले जाईल (या शब्दप्रयोगात, जोपर्यंत संदर्भ किंवा अर्थाच्या विरूद्ध नसेल, त्यांचे वारस, कायदेशीर प्रतिनिधी, मृत्युपत्र निष्पादक, प्रशासक आणि अभिहस्तांकिती यांचा समावेश असेल).\nआणि\n२. [खरेदीदाराचे नाव], वय वर्षे अंदाजे ___, राहणार [पत्ता], यापुढे 'खरेदीदार' म्हणून संबोधले जाईल (या शब्दप्रयोगात, जोपर्यंत संदर्भ किंवा अर्थाच्या विरूद्ध नसेल, त्यांचे वारस, कायदेशीर प्रतिनिधी, मृत्युपत्र निष्पादक, प्रशासक आणि अभिहस्तांकिती यांचा समावेश असेल).\nविक्रेता आणि खरेदीदार यांना एकत्रितपणे 'पक्षकार' म्हणून संबोधले जाईल.\n_________________

> **BLEU Score -** <br>
>> In this code first i have normalized translated marathi text then i have calculated bleu score
> **Prompt -**
Translate the following text into Marathi in simple, easy-to-understand Marathi.<br>
**Google Gemini 2.0 flash :** 0.28<br>
**Google Gemini 2.5 pro :** 0.23<br><br>
**Prompt -**
Translate the following text into Marathi keeping professional/legal tone.<br>
**Google Gemini 2.0 flash :** 0.25<br>
**Google Gemini 2.5 pro :** 0.20<br><br>
**Prompt -**
Translate the following text into Marathi exactly preserving legal terms.<br>
**Google Gemini 2.0 flash :** 0.24<br>
**Google Gemini 2.5 pro :** 0.23<br><br>
**Prompt -**
Translate the following English legal text into Marathi as a concise but legally accurate summary. Keep all rights, obligations, and \n clauses intact, but reduce redundancy. Suitable for executives or quick review.<br>
**Google Gemini 2.0 flash :** 0.23<br>
**Google Gemini 2.5 pro :** 0.24<br><br>
**Prompt -**
Translate the following English legal text into Marathi exactly preserving all legal terms and clause structures. \n Use precise legal terminology. Do not simplify or reword any clauses. Maintain numbering, punctuation, and formatting as in the original text.<br>
**Google Gemini 2.0 flash :** 0.23<br>
**Google Gemini 2.5 pro :** 0.20<br><br>
**Prompt -**
Translate the following English legal text into Marathi. Ensure all key legal terms like 'possession', 'Seller', 'Buyer', and 'government dues' \n are correctly translated. Maintain full context and structure. If a term has multiple possible translations, choose the one used in Indian legal documents.<br>
**Google Gemini 2.0 flash :** 0.22<br>
**Google Gemini 2.5 pro :** 0.22


In [9]:
def gemini_translate(text: str) -> str:
    response = gemini_client.models.generate_content(
        model="gemini-2.0-flash",
        contents=f"""
        Translate the following English legal text into Marathi in a formal legal register. 
        Use terminology consistent with Indian contracts and deeds. 
        Do not simplify or modernize the language. 
        Preserve original clause numbering, structure, and formatting. 
        Output only the translated document, nothing else.


        {text}""",
        config=types.GenerateContentConfig(
            temperature=0.0  # deterministic output
        )
    )
    return response.text


# ---------------- Run Translations ----------------
results = []

for prompt in prompts:  # ✅ only loop through prompts
    # --- Gemini ---
    gemini_translation = gemini_translate(f"{prompt}\n\n{english_text}")

    # --- BLEU Scores (Sentence-level, 0–1 scale) ---
    ref_tokens = reference_translation.split()

    gemini_bleu = sentence_bleu(
        [ref_tokens],
        gemini_translation.split(),
        weights=(0.25, 0.25, 0.25, 0.25)
    )

    results.append({
        "English Text": english_text[:100] + "...",
        "Prompt": prompt,
        "Google Gemini": gemini_translation,
        "Gemini BLEU": round(gemini_bleu, 2),
        "Reference Translation": reference_translation[:100] + "..."
    })

df = pd.DataFrame(results)
pd.set_option('display.max_colwidth', None)
display(HTML(df.to_html(escape=False)))


,English Text,Prompt,Google Gemini,Gemini BLEU,Reference Translation
0,"\nSALE DEED\nThis Sale Deed is executed on this ___ day of ____, 20 at [Place], Maharashtra.\nBY AND BE...",Translate the following text into Marathi exactly preserving legal terms.,"विक्रयपत्र\nहे विक्रयपत्र आज दिनांक ___ रोजी ____, इ.स. २०__ [ठिकाण], महाराष्ट्र येथे निष्पादित केले आहे.\nयामध्ये,\n1. [विक्रेत्याचे नाव], वय अंदाजे ___ वर्षे, रा. [पत्ता], यापुढे “विक्रेता” म्हणून संदर्भित (ज्यात संदर्भानुसार अन्यथा अर्थ अपेक्षित नसल्यास, त्यांचे वारस, कायदेशीर प्रतिनिधी, निष्पादक, प्रशासक आणि हक्क हस्तांतरिती यांचा समावेश असेल).\nआणि\n2. [खरेदीदाराचे नाव], वय अंदाजे ___ वर्षे, रा. [पत्ता], यापुढे “खरेदीदार” म्हणून संदर्भित (ज्यात संदर्भानुसार अन्यथा अर्थ अपेक्षित नसल्यास, त्यांचे वारस, कायदेशीर प्रतिनिधी, निष्पादक, प्रशासक आणि हक्क हस्तांतरिती यांचा समावेश असेल).\nविक्रेता आणि खरेदीदार यांना एकत्रितपणे “पक्षकार” म्हणून संबोधले जाईल.\n________________________________________\nWHEREAS (म्हणजे):\n1. विक्रेता भूखंडाचे संपूर्ण आणि कायदेशीर मालक आहेत, ज्या भूखंडाचा सर्व्हे नंबर [____] आहे, आणि त्याचे क्षेत्रफळ [____] चौ.मी./चौ.फूट आहे, जो [गाव/शहर, तालुका, जिल्हा, महाराष्ट्र] येथे स्थित आहे, यापुढे त्यास ""सदर मालमत्ता"" म्हणून संबोधले जाईल.\n2. विक्रेत्याने सदर मालमत्ता खरेदीदारास एकूण ₹[रक्कम] (रुपये [अक्षरात रक्कम]) च्या मोबदल्यात विक्री करण्याचे मान्य केले आहे.\n3. खरेदीदाराने सदर मालमत्ता विक्रेत्याकडून उपरोक्त मोबदल्यात, कोणत्याही भार,lien, दायित्व, गहाणखत किंवा न्यायालयीन खटल्यांपासून मुक्त खरेदी करण्याचे मान्य केले आहे.\n________________________________________\nआता हे দলিল সাক্ষ্য देते যে (NOW THIS DEED WITNESSETH AS FOLLOWS):\n1. विक्री आणि हस्तांतरण:\nविक्रेता, उपरोक्त नमूद केलेल्या मोबदल्याच्या बदल्यात, सदर मालमत्ता खरेदीदारास सर्व अधिकार, सुखाधिकार आणि तिच्याशी संलग्न असलेल्या जमीनीसह पूर्णपणे विक्री, हस्तांतरण आणि प्रदान करीत आहेत.\n2. मोबदला:\nएकूण विक्री मोबदला ₹[रक्कम] विक्रेत्याला खरेदीदाराकडून [भरणा करण्याची पद्धत: धनादेश/रोख/बँक हस्तांतरण] द्वारे प्राप्त झाला आहे, ज्याची पावती विक्रेता याद्वारे देत आहे.\n3. ताबा:\nविक्रेता याद्वारे सदर मालमत्तेचा प्रत्यक्ष आणि कायदेशीर ताबा या दस्ताच्या निष्पादनावर खरेदीदारास देत आहे.\n4. मालकी हक्क आणि हमी:\nविक्रेता खात्री देतात की सदर मालमत्ता सर्व भार, कायदेशीर विवाद, दावे किंवा गहाणखतांपासून मुक्त आहे. विक्रेता पुढे खरेदीदारास सदर मालमत्तेच्या मालकी हक्कामध्ये कोणत्याही दोषांमुळे उद्भवणाऱ्या कोणत्याही नुकसानी भरपाई देण्यास सहमत आहे.\n5. खर्च आणि नोंदणी:\n मुद्रांक शुल्क, नोंदणी शुल्क आणि इतर आनुषंगिक खर्चाशी संबंधित सर्व खर्च [खरेदीदार/विक्रेता यांच्यात परस्पर मान्यतेनुसार] द्वारे वहन केले जातील.\n6. कायदेशीर अधिकार:\nहे विक्रयपत्र भारतीय कायद्यानुसार आणि विशेषत: महाराष्ट्र मुद्रांक कायदा आणि मालमत्ता हस्तांतरण कायदा, १८८२ नुसार शासित आणि अन्वये लावले जाईल.\n________________________________________\nयाची साक्ष म्हणून, विक्रेता आणि खरेदीदार यांनी हे विक्रयपत्र वर नमूद केलेल्या दिवशी, महिन्यात आणि वर्षी स्वाक्षरी करून निष्पादित केले आहे.\nसही, शिक्का व सुपूर्द:\nविक्रेता:\n(सही) ___________________\nनाव: _______________________\nपत्ता: ____________________\nखरेदीदार:\n(सही) ___________________\nनाव: _______________________\nपत्ता: ____________________\n साक्षीदार:\n1. नाव: ___________________ सही: _______________ पत्ता: _______________\n2. नाव: ___________________ सही: _______________ पत्ता: _______________\n",0.25,विक्री करार\nहे विक्री करार २० तारखेला महाराष्ट्रातील [स्थान] येथे ____ दिवशी अंमलात आणले जात आहे .\nद...
1,"\nSALE DEED\nThis Sale Deed is executed on this ___ day of ____, 20 at [Place], Maharashtra.\nBY AND BE...","Translate the following text into Marathi in simple, easy-to-understand Marathi.","**विक्रयपत्र**\n\nहे विक्रयपत्र आज दिनांक ____, ____, २० रोजी [ठिकाण], महाराष्ट्र येथे निष्पादित केले जात आहे.\n\n**यांच्यामध्ये**\n\n1. [विक्रेत्याचे नाव], वय अंदाजे ___ वर्षे, रा. [पत्ता], यापुढे ""विक्रेता"" म्हणून संदर्भित (ज्यामध्ये, संदर्भास किंवा अर्थास विसंगत नसल्यास, त्यांचे वारस, कायदेशीर प्रतिनिधी, निष्पादक, प्रशासक आणि नियुक्त यांचा समावेश असेल).\n\nआणि\n\n2. [खरेदीदाराचे नाव], वय अंदाजे _

> **BLEU Score of Google Gemini 2.0 flash -** <br>
**Prompt -**
Translate the following text into Marathi exactly preserving legal terms.<br>
0.25<br><br>
**Prompt -**
Translate the following English legal text into Marathi as a concise but legally accurate summary. Keep all rights, obligations, and \n clauses intact, but reduce redundancy. Suitable for executives or quick review.<br>
0.25<br><br>
**Prompt -**
Translate the following text into Marathi keeping professional/legal tone.<br>
0.24<br><br>
**Prompt -**
Translate the following English legal text into Marathi exactly preserving all legal terms and clause structures. \n Use precise legal terminology. Do not simplify or reword any clauses. Maintain numbering, punctuation, and formatting as in the original text.<br>
0.23<br><br>
**Prompt -**
Translate the following English legal text into Marathi. Ensure all key legal terms like 'possession', 'Seller', 'Buyer', and 'government dues' \n are correctly translated. Maintain full context and structure. If a term has multiple possible translations, choose the one used in Indian legal documents.<br>
0.23<br><br>
**Prompt -**
Translate the following text into Marathi in simple, easy-to-understand Marathi.<br>
0.22

In [18]:
from nltk.translate.bleu_score import sentence_bleu

# ---------- Translation Functions (Only System Prompt) ----------
def groq_translate(text: str) -> str:
    return client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=[
            {"role": "system", "content": """
            You are a professional legal translator specializing in Indian contracts, deeds, and official agreements. 

Task: Translate the given English text into Marathi while adhering to the following rules:

1. Preserve all legal terminology exactly. Use standard Marathi legal terms used in Indian courts and official documents. 
2. Maintain the original clause numbering, bullet points, headings, and formatting. Do not alter the structure of the document.
3. Keep sentences formal, precise, and legally accurate. Do not simplify, modernize, or paraphrase.
4. Output only the translated document. Do not include explanations, summaries, or any additional text.
5. Ensure the translation is deterministic: repeated translations of the same text must yield the same result.

Translate the following English text into Marathi:

    """},
            {"role": "user", "content": text}
        ],
        temperature=0.0,
        max_tokens=1500,
    ).choices[0].message.content

def gpt_120b_translate(text: str) -> str:
    return client.chat.completions.create(
        model="openai/gpt-oss-120b",
        messages=[
            {"role": "system", "content": """
            You are a professional legal translator specializing in Indian contracts, deeds, and official agreements. 

Task: Translate the given English text into Marathi while adhering to the following rules:

1. Preserve all legal terminology exactly. Use standard Marathi legal terms used in Indian courts and official documents. 
2. Maintain the original clause numbering, bullet points, headings, and formatting. Do not alter the structure of the document.
3. Keep sentences formal, precise, and legally accurate. Do not simplify, modernize, or paraphrase.
4. Output only the translated document. Do not include explanations, summaries, or any additional text.
5. Ensure the translation is deterministic: repeated translations of the same text must yield the same result.

Translate the following English text into Marathi:

    """},
            {"role": "user", "content": text}
        ],
        temperature=0.0,
        max_tokens=1500,
    ).choices[0].message.content

In [19]:
# ---------------- Run Translations ----------------
results = []

for prompt in prompts:  # ✅ loop through prompts if you still need them
    groq_translation = groq_translate(f"{prompt}\n\n{english_text}")
    gpt120b_translation = gpt_120b_translate(f"{prompt}\n\n{english_text}")

    # --- Sentence BLEU (0–1 scale) ---
    groq_bleu = sentence_bleu([reference_translation.split()], groq_translation.split(), weights=(0.25, 0.25, 0.25, 0.25))
    gpt120b_bleu = sentence_bleu([reference_translation.split()], gpt120b_translation.split(), weights=(0.25, 0.25, 0.25, 0.25))

    results.append({
        "English Text": english_text[:100] + "...",
        "Prompt": prompt,
        "Groq LLaMA 3.3-70B": groq_translation,
        "Groq BLEU": round(groq_bleu, 2),
        "GPT-OSS 120B": gpt120b_translation,
        "GPT-OSS BLEU": round(gpt120b_bleu, 2),
        "Reference Translation": reference_translation[:100] + "..."
    })

df = pd.DataFrame(results)
pd.set_option('display.max_colwidth', None)
display(HTML(df.to_html(escape=False)))

,English Text,Prompt,Groq LLaMA 3.3-70B,Groq BLEU,GPT-OSS 120B,GPT-OSS BLEU,Reference Translation
0,"\nSALE DEED\nThis Sale Deed is executed on this ___ day of ____, 20 at [Place], Maharashtra.\nBY AND BE...",Translate the following text into Marathi exactly preserving legal terms.,"विक्री दाखला\nहा विक्री दाखला या ___ दिवशी, ____, 20 येथे [स्थळ], महाराष्ट्र येथे केला आहे.\nआणि\n1. [विक्रेत्याचे नाव], वय __ वर्षे, राहतो [पत्ता], येथपासून ""विक्रेता"" (या शब्दाचा अर्थ, संदर्भ किंवा त्याचा अर्थ विरुद्ध नसल्यास, त्याचे वारस, कायदेशीर प्रतिनिधी, नियुक्तीकर्ते, प्रशासक आणि हस्तांतरणकर्त्यांसह) म्हणून संबोधले जाईल.\nआणि\n2. [घटकाचे नाव], वय __ वर्षे, राहतो [पत्ता], येथपासून ""घटक"" (या शब्दाचा अर्थ, संदर्भ किंवा त्याचा अर्थ विरुद्ध नसल्यास, त्याचे वारस, कायदेशीर प्रतिनिधी, नियुक्तीकर्ते, प्रशासक आणि हस्तांतरणकर्त्यांसह) म्हणून संबोधले जाईल.\nविक्रेता आणि घटक हे सामूहिकपणे ""पक्ष"" म्हणून संबोधले जातात.\n________________________________________\nयाविषयी:\n1. विक्रेता हा सर्व्हे नंबर [____] असलेल्या जागेचा निस्संशय आणि कायदेशीर मालक आहे, ज्याचे क्षेत्रफळ [____] चौरस मी. / चौरस फूट आहे, जी [गाव/शहर, तालuka, जिल्हा, महाराष्ट्र] येथे स्थित आहे, याला ""सांगितलेली मालमत्ता"" म्हणून संबोधले जाईल.\n2. विक्रेत्याने घटकाला सांगितलेल्या मालमत्तेचे विक्री मूल्य ₹ [परिशिष्ट] (रुपये [परिशिष्ट शब्द]) या एकूण विचाराने करण्यास सहमती दर्शविली आहे.\n3. घटकाने विक्रेत्याकडून सांगितलेली मालमत्ता वich मुक्त, सर्व बंधने, हक्क, शुल्क, हिपोटेक्स किंवा खटला निरपेक्ष राखून घेण्यास सहमती दर्शविली आहे.\n________________________________________\nआता हा दाखला साक्ष देतो:\n1. विक्री आणि हस्तांतरण:\nविक्रेत्याने वर नमूद केलेल्या विचाराच्या बदल्यात, सांगितलेली मालमत्ता घटकाला पूर्णपणे विकतो, हस्तांतरित करतो आणि सोपवितो, त्यासह सर्व हक्क, सुविधा आणि संलग्नक.\n2. विचार:\n₹ [परिशिष्ट] या एकूण विक्री विचाराची रक्कम घटकाकडून विक्रेत्याला [पेमेंट मोड: चेक/कॅश/बँक हस्तांतरण] मार्गाने मिळाली आहे, ज्याची पावती विक्रेत्याने या दाखल्याने मान्य केली आहे.\n3. ताबा:\nविक्रेत्याने या दाखल्याच्या निर्मितीच्या वेळी सांगितलेल्या मालमत्तेचा भौतिक आणि कायदेशीर ताबा घटकाला सोपविला आहे.\n4. शीर्षक आणि हमी:\nविक्रेत्याने ashवाधित केले आहे की सांगितलेली मालमत्ता सर्व बंधने, कायदेशीर विवाद, दावे किंवा हिपोटेक्स मुक्त आहे. विक्रेत्याने आणखी अशी सहमती दर्शविली आहे की ते सांगितलेल्या मालमत्तेच्या शीर्षणातील कोणत्याही दोषामुळे घटकाला होणारी कोणतीही हानी भरून देतील.\n5. खर्च आणि नोंदणी:\nशुल्कबाबतची सर्व खर्च, नोंदणी शुल्क आणि इतर आकस्मिक खर्च [खरीददार/विक्रेता दोघांनी सहमती दर्शविली] असल्यास वहन करण्यात येतील.\n6. शासन कायदा:\nहा विक्री दाखला भारतीय कायद्यांनुसार आणि विशेषतः महाराष्ट्र स्टॅम्प अधिनियम आणि संपत्ती हस्तांतरण अधिनियम, 1882 नुसार शासित केला जाईल आणि अर्थ लावला जाईल.\n________________________________________\nयाची साक्ष इथे, विक्रेता आणि घटक यांनी या दakhदखल्यावर सही केली आहे.\nसही केलं, शिक्का केला आणि सांगितलेल्या दिवशी, महिन्यात आणि वर्षी या दाखल्यावर:\nविक्रेता:\n(सही) ___________________\nनाव: _______________________\nपत्ता: ____________________\nघटक:\n(सही) ___________________\nनाव: _______________________\nपत्ता: ____________________\nसाक्षिदार:\n1. नाव: ___________________ सही: _______________ पत्ता: _______________\n2. नाव: ___________________ सही: _______________ पत्ता: _______________",0.13,"**विक्री करारपत्र** \nहे विक्री करारपत्र ___ दिनांक ____, 20 रोजी [स्थळ], महाराष्ट्र येथे अंमलात आणले जाते. \n\n**दरम्यान** \n1. [विक्रेता नाव], वय अंदाजे ___ वर्षे, राहते [पत्ता], यांना पुढे “**विक्रेता**” (ज्याचा अर्थ, संदर्भ व अर्थानुसार, त्याचे वारस, कायदेशीर प्रतिनिधी, कार्यकारी, प्रशासक व हस्तांतरित व्यक्ती यांचा समावेश होईल) म्हणून ओळखले जाईल. \n2. [खरेदीदार नाव], वय अंदाजे ___ वर्षे, राहते [पत्ता], यांना पुढे “**खरेदीदार**” (ज्याचा अर्थ, संदर्भ व अर्थानुसार, त्याचे वारस, कायदेशीर प्रतिनिधी, कार्यकारी, प्रशासक व हस्तांतरित व्यक्ती यांचा समावेश होईल) म्हणून ओळखले जाईल. \n\nविक्रेता व खरेदीदार यांना एकत्रितपणे “**पक्ष**” म्हणून ओळखले जाईल. \n\n________________________________________ \n\n**ज्यामुळे:** \n1. विक्रेत्याचा सर्वे क्रमांक [____] असलेला, [____] वर्ग मी./वर्ग फूट क्षेत्रफ

> **BLEU Score -** <br>
**Prompt -**
Translate the following text into Marathi exactly preserving legal terms.<br>
**Groq LLaMA 3.3-70B :** 0.13<br>
**GPT-OSS 120B :** 0.10<br><br>
**Prompt -**
Translate the following text into Marathi in simple, easy-to-understand Marathi.<br>
**Groq LLaMA 3.3-70B :** 0.11<br>
**GPT-OSS 120B :** 0.10<br><br>
**Prompt -**
Translate the following text into Marathi keeping professional/legal tone.<br>
**Groq LLaMA 3.3-70B :** 0.10<br>
**GPT-OSS 120B :** 0.08<br><br>
**Prompt -**
Translate the following English legal text into Marathi exactly preserving all legal terms and clause structures. \n Use precise legal terminology. Do not simplify or reword any clauses. Maintain numbering, punctuation, and formatting as in the original text.<br>
**Groq LLaMA 3.3-70B :** 0.14<br>
**GPT-OSS 120B :** 0.05<br><br>
**Prompt -**
Translate the following English legal text into Marathi as a concise but legally accurate summary. Keep all rights, obligations, and \n clauses intact, but reduce redundancy. Suitable for executives or quick review.<br>
**Groq LLaMA 3.3-70B :** 0.11<br>
**GPT-OSS 120B :** 0.08<br><br>
**Prompt -** 
Translate the following English legal text into Marathi. Ensure all key legal terms like 'possession', 'Seller', 'Buyer', and 'government dues' \n are correctly translated. Maintain full context and structure. If a term has multiple possible translations, choose the one used in Indian legal documents.<br>
**Groq LLaMA 3.3-70B :** 0.14<br>
**GPT-OSS 120B :** 0.11

In [20]:
# ---------------- Run Translations ----------------
results = []

for prompt in prompts:  # ✅ loop through prompts if you still need them
    groq_translation = groq_translate(f"{prompt}\n\n{english_text}")
    gpt120b_translation = gpt_120b_translate(f"{prompt}\n\n{english_text}")

    # --- Sentence BLEU (0–1 scale) ---
    groq_bleu = sentence_bleu([reference_translation.split()], groq_translation.split(), weights = (0.1, 0.2, 0.3, 0.4))
    gpt120b_bleu = sentence_bleu([reference_translation.split()], gpt120b_translation.split(), weights = (0.1, 0.2, 0.3, 0.4))

    results.append({
        "English Text": english_text[:100] + "...",
        "Prompt": prompt,
        "Groq LLaMA 3.3-70B": groq_translation,
        "Groq BLEU": round(groq_bleu, 2),
        "GPT-OSS 120B": gpt120b_translation,
        "GPT-OSS BLEU": round(gpt120b_bleu, 2),
        "Reference Translation": reference_translation[:100] + "..."
    })

df = pd.DataFrame(results)
pd.set_option('display.max_colwidth', None)
display(HTML(df.to_html(escape=False)))

,English Text,Prompt,Groq LLaMA 3.3-70B,Groq BLEU,GPT-OSS 120B,GPT-OSS BLEU,Reference Translation
0,"\nSALE DEED\nThis Sale Deed is executed on this ___ day of ____, 20 at [Place], Maharashtra.\nBY AND BE...",Translate the following text into Marathi exactly preserving legal terms.,"विक्री करार\nहा विक्री करार दिनांक ___ ___ 20, रोजी [स्थान], महाराष्ट्र येथे केला आहे.\nपरस्परांमध्ये \n1. [विक्रेत्याचे नाव], वय ___ वर्षे, पत्ता [पत्ता], यांना यापुढे ""विक्रेता"" असे संबोधले जाईल (ज्याचा अर्थ असa की विक्रेत्याचे वारस, कायदेशीर प्रतिनिधी, एग्झीक्यूटर्स, एडमिनिस्ट्रेटर्स आणि असाईन्स यांचा समावेश असेल, परंतु संदर्भ किंवा अशा अर्थाच्या विरोधात असल्यास).\nआणि\n2. [खरेदीदाराचे नाव], वय ___ वर्षे, पत्ता [पत्ता], यांना यापुढे ""खरेदीदार"" असे संबोधले जाईल (ज्याचा अर्थ असा की खरेदीदाराचे वारस, कायदेशीर प्रतिनिधी, एग्झीक्यूटर्स, एडमिनिस्ट्रेटर्स आणि असाईन्स यांचा समावेश असेल, परंतु संदर्भ किंवा अशा अर्थाच्या विरोधात असल्यास).\nविक्रेता आणि खरेदीदार हे सामूहिकपणे ""पक्ष"" म्हणून संबोधले जातात.\n________________________________________\nयासाठी:\n1. विक्रेता मालकी हक्काचा एकमेव आणि कायदेशीर मालक आहे जमिनीचा प्लॉट सर्व्हे नं. [____], क्षेत्रफळ [____] चौरस मी./चौरस फूट, [गाव/शहर, तालुका, जिल्हा, महाराष्ट्र] येथे स्थित, याला ""सांगितलेली मालमत्ता"" म्हणून संबोधले जाईल.\n2. विक्रेत्याने सांगितलेली मालमत्ता खरेदीदाराला एकूण ₹[रकम] (रुपये [रकमेचे शब्द]) या मोबदल्यात विकण्यास सहमती दर्शविली आहे.\n3. खरेदीदाराने सांगितलेली मालमत्ता विक्रेत्याकडून वर नमूद केलेल्या मोबदल्यासाठी कोणत्याही भार, प्रतिबंध, छळ, हप्ते, किंवा खटल्यापासून मुक्त असल्याचे सहमती दर्शविली आहे.\n________________________________________\nअता हा करार अनुवादित आहे:\n1. विक्री आणि हस्तांतरण:\nविक्रेत्याने वर नमूद केलेल्या मोबदल्यासाठी, सांगितलेली मालमत्ता खरेदीदाराला पूर्णपणे विक्री, हस्तांतरण आणि सुपुर्दगी करतो, त्यासह सर्व हक्क, उपयोजन आणि संलग्नक.\n2. मोबदला:\nएकूण विक्री मोबदला ₹[रकम] विक्रेत्याने खरेदीदाराकडून [पेमेंटचे प्रकार: चेक/नकद/बँक हस्तांतरण] द्वारे मिळालेला आहे, ज्याची पावती विक्रेत्याने येथे दिली आहे.\n3. ताबा:\nविक्रेत्याने सांगितलेली मालमत्ता या कराराच्या निर्णयाच्या वेळी खरेदीदाराला भौतिक आणि कायदेशीर ताबा सुपुर्द केला आहे.\n4. मालकी हक्क आणि वॉरंटी:\nविक्रेत्याने अशी खात्री दिली आहे की सांगितलेली मालमत्ता कोणत्याही भार, कायदेशीर विवाद, दावे किंवा हप्त्यांपासून मुक्त आहे. विक्रेत्याने सांगितलेल्या मालमत्तेच्या मालकी हक्काच्या कोणत्याही दोषामुळे होणार्या नुकसानीपासून खरेदीदाराला नुकसानभरपाई देण्यास सहमती दर्शविली आहे.\n5. व्यय आणि नोंदणी:\nस्टॅम्प शुल्क, नोंदणी शुल्क आणि अन्य अप्रत्यक्ष व्यय यासंबंधित सर्व व्यय [खरेदीदार/विक्रेता परस्पर सामंजस्याने] सहन करतील.\n6. शासन कायदा:\nहा विक्री करार भारतीय कायद्यांनुसार आणि विशेषतः महाराष्ट्र स्टॅम्प कायदा आणि 1882 चा मालमत्ता हस्तांतरण कायद्यानुसार शासित आणि व्याख्या केली जाईल.\n________________________________________\nयाची साक्ष देता:\nविक्रेता आणि खरेदीदाराने या करारावर वर नमूद केलेल्या दिनांक, महिना आणि वर्षी स्वाक्षरी केली आहे.\nस्वाक्षरी, सील आणि सुपुर्द केले:\nविक्रेता:\n(स्वाक्षरी) ___________________\nनाव: _______________________\nपत्ता: ____________________\nखरेदीदार:\n(स्वाक्षरी) ___________________\nनाव: _______________________\nपत्ता: ____________________\nसाक्षीदार:\n1. नाव: ___________________",0.09,"SALE DEED \nहे विक्री करारपत्र ____ दिनांक ____, 20 रोजी [Place], महाराष्ट्र येथे अंमलात आणले गेले आहे. \n\nBY AND BETWEEN \n\n1. [Seller Name], वय सुमारे ___ वर्षे, राहते [Address], पुढे “विक्रेता” म्हणून उल्लेखित (ज्याचा अर्थ, संदर्भ किंवा अर्थाच्या विरोधात नसल्यास, त्याचे वारस, कायदेशीर प्रतिनिधी, कार्यकारी, प्रशासक व हस्तांतरित व्यक्ती यांचा समावेश होईल). \nAND \n2. [Buyer Name], वय सुमारे ___ वर्षे, राहते [Address], पुढे “खरेदीदार” म्हणून उल्लेखित (ज्याचा अर्थ, संदर्भ किंवा अर्थाच्या विरोधात नसल्यास, त्याचे वारस, कायदेशीर प्रतिनिधी, कार्यकारी, प्रशासक व हस्तांतरित व्यक्ती यांचा समावेश होईल). \n\nविक्रेता व खरेदीदार यांना एकत्रितपणे “पक्ष” म्हणून उल्लेखित केले आहे. \n\n________________________________________ \n\nWHEREAS: \n\n1. विक्रेत्याचा सर्वसाधारण व कायदेशीर मालकीचा भूखंड, सर्वे क्रमांक [____], क्षेत्रफळ 

> BLEU Score - <br>
**Prompt -**
Translate the following English legal text into Marathi. Ensure all key legal terms like 'possession', 'Seller', 'Buyer', and 'government dues' \n are correctly translated. Maintain full context and structure. If a term has multiple possible translations, choose the one used in Indian legal documents.
<br>
**Groq LLaMA 3.3-70B :** 0.08<br>
**GPT-OSS 120B :** 0.04<br><br>
**Prompt -**
Translate the following English legal text into Marathi as a concise but legally accurate summary. Keep all rights, obligations, and \n clauses intact, but reduce redundancy. Suitable for executives or quick review.
<br>
**Groq LLaMA 3.3-70B :** 0.10<br>
**GPT-OSS 120B :** 0.04<br><br>
**Prompt -** 
Translate the following English legal text into Marathi exactly preserving all legal terms and clause structures. \n Use precise legal terminology. Do not simplify or reword any clauses. Maintain numbering, punctuation, and formatting as in the original text.
<br>
**Groq LLaMA 3.3-70B :** 0.10<br>
**GPT-OSS 120B :** 0.06<br><br>
**Prompt -** 
Translate the following text into Marathi keeping professional/legal tone.
<br>
**Groq LLaMA 3.3-70B :** 0.12<br>
**GPT-OSS 120B :** 0.07<br><br>
**Prompt -**
Translate the following text into Marathi in simple, easy-to-understand Marathi.
<br>
**Groq LLaMA 3.3-70B :** 0.14<br>
**GPT-OSS 120B :** 0.13<br><br>
**Prompt -**
Translate the following text into Marathi exactly preserving legal terms.
<br>
**Groq LLaMA 3.3-70B :** 0.09<br>
**GPT-OSS 120B :** 0.04

In [22]:
def gemini_translate(text: str) -> str:
    response = gemini_client.models.generate_content(
        model="gemini-2.0-flash",
        contents=f"""
        Translate the following English legal text into Marathi in a formal legal register. 
        Use terminology consistent with Indian contracts and deeds. 
        Do not simplify or modernize the language. 
        Preserve original clause numbering, structure, and formatting. 
        Output only the translated document, nothing else.


        {text}""",
        config=types.GenerateContentConfig(
            temperature=0.0  # deterministic output
        )
    )
    return response.text


# ---------------- Run Translations ----------------
results = []

for prompt in prompts:  # ✅ only loop through prompts
    # --- Gemini ---
    gemini_translation = gemini_translate(f"{prompt}\n\n{english_text}")

    # --- BLEU Scores (Sentence-level, 0–1 scale) ---
    ref_tokens = reference_translation.split()

    gemini_bleu = sentence_bleu(
        [ref_tokens],
        gemini_translation.split(),
        weights = (0.1, 0.2, 0.3, 0.4)
    )

    results.append({
        "English Text": english_text[:100] + "...",
        "Prompt": prompt,
        "Google Gemini": gemini_translation,
        "Gemini BLEU": round(gemini_bleu, 2),
        "Reference Translation": reference_translation[:100] + "..."
    })

df = pd.DataFrame(results)
pd.set_option('display.max_colwidth', None)
display(HTML(df.to_html(escape=False)))


,English Text,Prompt,Google Gemini,Gemini BLEU,Reference Translation
0,"\nSALE DEED\nThis Sale Deed is executed on this ___ day of ____, 20 at [Place], Maharashtra.\nBY AND BE...",Translate the following text into Marathi exactly preserving legal terms.,"विक्रयपत्र\nहे विक्रयपत्र आज दिनांक ___ रोजी ____, इ.स. २०__ [ठिकाण], महाराष्ट्र येथे निष्पादित केले आहे.\nयामध्ये,\n1. [विक्रेत्याचे नाव], वय अंदाजे ___ वर्षे, रा. [पत्ता], यापुढे “विक्रेता” म्हणून संदर्भित (ज्यात संदर्भानुसार अन्यथा अर्थ अपेक्षित नसल्यास, त्यांचे वारस, कायदेशीर प्रतिनिधी, निष्पादक, प्रशासक आणि हक्क हस्तांतरिती यांचा समावेश असेल).\nआणि\n2. [खरेदीदाराचे नाव], वय अंदाजे ___ वर्षे, रा. [पत्ता], यापुढे “खरेदीदार” म्हणून संदर्भित (ज्यात संदर्भानुसार अन्यथा अर्थ अपेक्षित नसल्यास, त्यांचे वारस, कायदेशीर प्रतिनिधी, निष्पादक, प्रशासक आणि हक्क हस्तांतरिती यांचा समावेश असेल).\nविक्रेता आणि खरेदीदार यांना एकत्रितपणे “पक्षकार” म्हणून संबोधले जाईल.\n________________________________________\nWHEREAS (म्हणजे):\n1. विक्रेता भूखंडाचे संपूर्ण आणि कायदेशीर मालक आहेत, ज्या भूखंडाचा सर्व्हे नंबर [____] आहे, आणि त्याचे क्षेत्रफळ [____] चौ.मी./चौ.फूट आहे, जो [गाव/शहर, तालुका, जिल्हा, महाराष्ट्र] येथे स्थित आहे, यापुढे त्यास ""सदर मालमत्ता"" म्हणून संबोधले जाईल.\n2. विक्रेत्याने सदर मालमत्ता खरेदीदारास एकूण ₹[रक्कम] (रुपये [अक्षरात रक्कम]) च्या मोबदल्यात विक्री करण्याचे मान्य केले आहे.\n3. खरेदीदाराने सदर मालमत्ता विक्रेत्याकडून उपरोक्त मोबदल्यात, कोणत्याही भार,lien, दायित्व, गहाणखत किंवा न्यायालयीन खटल्यांपासून मुक्त खरेदी करण्याचे मान्य केले आहे.\n________________________________________\nआता हे দলিল সাক্ষ্য देते যে (NOW THIS DEED WITNESSETH AS FOLLOWS):\n1. विक्री आणि हस्तांतरण:\nविक्रेता, उपरोक्त नमूद केलेल्या मोबदल्याच्या बदल्यात, सदर मालमत्ता खरेदीदारास सर्व अधिकार, सुखाधिकार आणि तिच्याशी संलग्न असलेल्या जमीनीसह पूर्णपणे विक्री, हस्तांतरण आणि प्रदान करीत आहेत.\n2. मोबदला:\nएकूण विक्री मोबदला ₹[रक्कम] विक्रेत्याला खरेदीदाराकडून [भरणा करण्याची पद्धत: धनादेश/रोख/बँक हस्तांतरण] द्वारे प्राप्त झाला आहे, ज्याची पावती विक्रेता याद्वारे देत आहे.\n3. ताबा:\nविक्रेता याद्वारे सदर मालमत्तेचा प्रत्यक्ष आणि कायदेशीर ताबा या दस्ताच्या निष्पादनावर खरेदीदारास देत आहे.\n4. मालकी हक्क आणि हमी:\nविक्रेता खात्री देतात की सदर मालमत्ता सर्व भार, कायदेशीर विवाद, दावे किंवा गहाणखतांपासून मुक्त आहे. विक्रेता पुढे खरेदीदारास सदर मालमत्तेच्या मालकी हक्कामध्ये कोणत्याही दोषांमुळे उद्भवणाऱ्या कोणत्याही नुकसानी भरपाई देण्यास सहमत आहे.\n5. खर्च आणि नोंदणी:\n मुद्रांक शुल्क, नोंदणी शुल्क आणि इतर आनुषंगिक खर्चाशी संबंधित सर्व खर्च [खरेदीदार/विक्रेता यांच्यात परस्पर मान्यतेनुसार] द्वारे वहन केले जातील.\n6. कायदेशीर अधिकार:\nहे विक्रयपत्र भारतीय कायद्यानुसार आणि विशेषत: महाराष्ट्र मुद्रांक कायदा आणि मालमत्ता हस्तांतरण कायदा, १८८२ नुसार शासित आणि अन्वये लावले जाईल.\n________________________________________\nयाची साक्ष म्हणून, विक्रेता आणि खरेदीदार यांनी हे विक्रयपत्र वर नमूद केलेल्या दिवशी, महिन्यात आणि वर्षी स्वाक्षरी करून निष्पादित केले आहे.\nसही, शिक्का व सुपूर्द:\nविक्रेता:\n(सही) ___________________\nनाव: _______________________\nपत्ता: ____________________\nखरेदीदार:\n(सही) ___________________\nनाव: _______________________\nपत्ता: ____________________\n साक्षीदार:\n1. नाव: ___________________ सही: _______________ पत्ता: _______________\n2. नाव: ___________________ सही: _______________ पत्ता: _______________\n",0.20,विक्री करार\nहे विक्री करार २० तारखेला महाराष्ट्रातील [स्थान] येथे ____ दिवशी अंमलात आणले जात आहे .\nद...
1,"\nSALE DEED\nThis Sale Deed is executed on this ___ day of ____, 20 at [Place], Maharashtra.\nBY AND BE...","Translate the following text into Marathi in simple, easy-to-understand Marathi.","**विक्रयपत्र**\n\nहे विक्रयपत्र आज दिनांक ____, ____, २० रोजी [ठिकाण], महाराष्ट्र येथे निष्पादित केले जात आहे.\n\nयाद्वारे:\n\n1. [विक्रेत्याचे नाव], वय अंदाजे ___ वर्षे, रा. [पत्ता], यापुढे ""विक्रेता"" म्हणून संदर्भित (ज्यामध्ये, संदर्भास किंवा अर्थास प्रतिकूल नसल्यास, त्यांचे वारस, कायदेशीर प्रतिनिधी, निष्पादक, प्रशासक आणि नियुक्त यांचा समावेश असेल).\n\nआणि\n\n2. [खरेदीदाराचे नाव], वय अंदाजे ___ वर

> This code takes english text as input and translate it into marathi using Gemini 2.0 flash model and calculates bleu score using sentence bleu ...In this i have given more importance to the 4-gram wieght but by this i have observed that if i give more importance to 4-gram the accuracy decreases<br>
0.0 – 0.2 → Poor translation (many mismatches).<br>
0.2 – 0.4 → Understandable but with errors.<br>
0.4 – 0.6 → Good translation, but not perfect.<br>
0.6 – 0.8 → Very good translation, close to human quality.<br>
0.8 – 1.0 → Excellent / near-human translation (rare in practice).<br>

>> **How BLEU score is being calculated ?** <br>in this code the marathi generated text and reference text is splited into words and then unigram, bigram, trigram, 4-gram are compared how many times it apears in reference text , more importance is given to 4-gram words in this code and if the generated text is given penalty if it is shorter than reference text then it returns score between 0 to 1.<br>

> **BLEU Score of Google Gemini 2.0 flash -** <br>
**Prompt -**
Translate the following text into Marathi exactly preserving legal terms<br>
Score - 0.20<br><br>
**Prompt -**
Translate the following text into Marathi in simple, easy-to-understand Marathi.<br>
Score - 0.21<br><br>
**Prompt -**
Translate the following text into Marathi keeping professional/legal tone.<br>
Score - 0.19<br><br>
**Prompt -**
Translate the following English legal text into Marathi exactly preserving all legal terms and clause structures. \n Use precise legal terminology. Do not simplify or reword any clauses. Maintain numbering, punctuation, and formatting as in the original text.<br>
Score - 0.17<br><br>
**Prompt -**
Translate the following English legal text into Marathi as a concise but legally accurate summary. Keep all rights, obligations, and \n clauses intact, but reduce redundancy. Suitable for executives or quick review.<br>
Score - 0.20<br><br>
**Prompt -**
Translate the following English legal text into Marathi. Ensure all key legal terms like 'possession', 'Seller', 'Buyer', and 'government dues' \n are correctly translated. Maintain full context and structure. If a term has multiple possible translations, choose the one used in Indian legal documents.<br>
Score - 0.15

In [27]:
def gemini_translate(text: str) -> str:
    response = gemini_client.models.generate_content(
        model="gemini-2.0-flash",
        contents=f"""
        Translate the following English legal text into Marathi in a formal legal register. 
        Use terminology consistent with Indian contracts and deeds. 
        Do not simplify or modernize the language. 
        Preserve original clause numbering, structure, and formatting. 
        Output only the translated document, nothing else.


        {text}""",
        config=types.GenerateContentConfig(
            temperature=0.0  # deterministic output
        )
    )
    return response.text


# ---------------- Run Translations ----------------
results = []

for prompt in prompts:  # ✅ only loop through prompts
    # --- Gemini ---
    gemini_translation = gemini_translate(f"{prompt}\n\n{english_text}")

    # --- BLEU Scores (Sentence-level, 0–1 scale) ---
    ref_tokens = reference_translation.split()

    gemini_bleu = sentence_bleu(
        [ref_tokens],
        gemini_translation.split(),
        weights = (1.0, 0.1, 0.1, 0.1)
    )

    results.append({
        "English Text": english_text[:100] + "...",
        "Prompt": prompt,
        "Google Gemini": gemini_translation,
        "Gemini BLEU": round(gemini_bleu, 2),
        "Reference Translation": reference_translation[:100] + "..."
    })

df = pd.DataFrame(results)
pd.set_option('display.max_colwidth', None)
display(HTML(df.to_html(escape=False)))

,English Text,Prompt,Google Gemini,Gemini BLEU,Reference Translation
0,"\nSALE DEED\nThis Sale Deed is executed on this ___ day of ____, 20 at [Place], Maharashtra.\nBY AND BE...",Translate the following text into Marathi exactly preserving legal terms.,"विक्रयपत्र\nहे विक्रयपत्र आज दिनांक ___ रोजी ____, इ.स. २०__ [ठिकाण], महाराष्ट्र येथे निष्पादित केले आहे.\nयामध्ये,\n1. [विक्रेत्याचे नाव], वय अंदाजे ___ वर्षे, रा. [पत्ता], यापुढे “विक्रेता” म्हणून संदर्भित (ज्यात संदर्भानुसार अन्यथा अर्थ अपेक्षित नसल्यास, त्यांचे वारस, कायदेशीर प्रतिनिधी, निष्पादक, प्रशासक आणि हक्क हस्तांतरिती यांचा समावेश असेल).\nआणि\n2. [खरेदीदाराचे नाव], वय अंदाजे ___ वर्षे, रा. [पत्ता], यापुढे “खरेदीदार” म्हणून संदर्भित (ज्यात संदर्भानुसार अन्यथा अर्थ अपेक्षित नसल्यास, त्यांचे वारस, कायदेशीर प्रतिनिधी, निष्पादक, प्रशासक आणि हक्क हस्तांतरिती यांचा समावेश असेल).\nविक्रेता आणि खरेदीदार यांना एकत्रितपणे “पक्षकार” म्हणून संबोधले जाईल.\n________________________________________\nWHEREAS (म्हणजे):\n1. विक्रेता भूखंडाचे संपूर्ण आणि कायदेशीर मालक आहेत, ज्या भूखंडाचा सर्व्हे नंबर [____] आहे, आणि त्याचे क्षेत्रफळ [____] चौ.मी./चौ.फूट आहे, जो [गाव/शहर, तालुका, जिल्हा, महाराष्ट्र] येथे स्थित आहे, यापुढे त्यास ""सदर मालमत्ता"" म्हणून संबोधले जाईल.\n2. विक्रेत्याने सदर मालमत्ता खरेदीदारास एकूण ₹[रक्कम] (रुपये [अक्षरात रक्कम]) च्या मोबदल्यात विक्री करण्याचे मान्य केले आहे.\n3. खरेदीदाराने सदर मालमत्ता विक्रेत्याकडून उपरोक्त मोबदल्यात, कोणत्याही भार,lien, दायित्व, गहाणखत किंवा न्यायालयीन खटल्यांपासून मुक्त खरेदी करण्याचे मान्य केले आहे.\n________________________________________\nआता हे দলিল সাক্ষ্য देते যে (NOW THIS DEED WITNESSETH AS FOLLOWS):\n1. विक्री आणि हस्तांतरण:\nविक्रेता, उपरोक्त नमूद केलेल्या मोबदल्याच्या बदल्यात, सदर मालमत्ता खरेदीदारास सर्व अधिकार, सुखाधिकार आणि तिच्याशी संलग्न असलेल्या जमीनीसह पूर्णपणे विक्री, हस्तांतरण आणि प्रदान करीत आहेत.\n2. मोबदला:\nएकूण विक्री मोबदला ₹[रक्कम] विक्रेत्याला खरेदीदाराकडून [भरणा करण्याची पद्धत: धनादेश/रोख/बँक हस्तांतरण] द्वारे प्राप्त झाला आहे, ज्याची पावती विक्रेता याद्वारे देत आहे.\n3. ताबा:\nविक्रेता याद्वारे सदर मालमत्तेचा प्रत्यक्ष आणि कायदेशीर ताबा या दस्ताच्या निष्पादनावर खरेदीदारास देत आहे.\n4. मालकी हक्क आणि हमी:\nविक्रेता खात्री देतात की सदर मालमत्ता सर्व भार, कायदेशीर विवाद, दावे किंवा गहाणखतांपासून मुक्त आहे. विक्रेता पुढे खरेदीदारास सदर मालमत्तेच्या मालकी हक्कामध्ये कोणत्याही दोषांमुळे उद्भवणाऱ्या कोणत्याही नुकसानी भरपाई देण्यास सहमत आहे.\n5. खर्च आणि नोंदणी:\n मुद्रांक शुल्क, नोंदणी शुल्क आणि इतर आनुषंगिक खर्चाशी संबंधित सर्व खर्च [खरेदीदार/विक्रेता यांच्यात परस्पर मान्यतेनुसार] द्वारे वहन केले जातील.\n6. कायदेशीर अधिकार:\nहे विक्रयपत्र भारतीय कायद्यानुसार आणि विशेषत: महाराष्ट्र मुद्रांक कायदा आणि मालमत्ता हस्तांतरण कायदा, १८८२ नुसार शासित आणि अन्वये लावले जाईल.\n________________________________________\nयाची साक्ष म्हणून, विक्रेता आणि खरेदीदार यांनी हे विक्रयपत्र वर नमूद केलेल्या दिवशी, महिन्यात आणि वर्षी स्वाक्षरी करून निष्पादित केले आहे.\nसही, शिक्का व सुपूर्द:\nविक्रेता:\n(सही) ___________________\nनाव: _______________________\nपत्ता: ____________________\nखरेदीदार:\n(सही) ___________________\nनाव: _______________________\nपत्ता: ____________________\n साक्षीदार:\n1. नाव: ___________________ सही: _______________ पत्ता: _______________\n2. नाव: ___________________ सही: _______________ पत्ता: _______________\n",0.34,विक्री करार\nहे विक्री करार २० तारखेला महाराष्ट्रातील [स्थान] येथे ____ दिवशी अंमलात आणले जात आहे .\nद...
1,"\nSALE DEED\nThis Sale Deed is executed on this ___ day of ____, 20 at [Place], Maharashtra.\nBY AND BE...","Translate the following text into Marathi in simple, easy-to-understand Marathi.","**विक्रयपत्र**\n\nहे विक्रयपत्र आज दिनांक ____, ____, २० रोजी [ठिकाण], महाराष्ट्र येथे निष्पादित केले जात आहे.\n\nयाद्वारे:\n\n1. [विक्रेत्याचे नाव], वय अंदाजे ___ वर्षे, रा. [पत्ता], यापुढे ""विक्रेता"" म्हणून संदर्भित (ज्यामध्ये, संदर्भास किंवा अर्थास प्रतिकूल नसल्यास, त्यांचे वारस, कायदेशीर प्रतिनिधी, निष्पादक, प्रशासक आणि नियुक्त यांचा समावेश असेल).\n\nआणि\n\n2. [खरेदीदाराचे नाव], वय अंदाजे ___ वर

> This code takes english text as input and translate it into marathi using Gemini 2.0 flash model and calculates bleu score using sentence bleu<br>
0.0 – 0.2 → Poor translation (many mismatches).<br>
0.2 – 0.4 → Understandable but with errors.<br>
0.4 – 0.6 → Good translation, but not perfect.<br>
0.6 – 0.8 → Very good translation, close to human quality.<br>
0.8 – 1.0 → Excellent / near-human translation (rare in practice).<br>
>> **BLEU Score of Google Gemini 2.0 flash -** <br>
**Prompt -**
Translate the following text into Marathi exactly preserving legal terms.<br>
0.34<br><br>
**Prompt -**
Translate the following text into Marathi in simple, easy-to-understand Marathi.<br>
0.38<br><br>
**Prompt -**
Translate the following text into Marathi keeping professional/legal tone.<br>
0.31<br><br>
**Prompt -**
Translate the following English legal text into Marathi exactly preserving all legal terms and clause structures. \n Use precise legal terminology. Do not simplify or reword any clauses. Maintain numbering, punctuation, and formatting as in the original text.<br>
0.32<br><br>
**Prompt -** 
Translate the following English legal text into Marathi as a concise but legally accurate summary. Keep all rights, obligations, and \n clauses intact, but reduce redundancy. Suitable for executives or quick review.<br>
0.30<br><br>
**Prompt -**
Translate the following English legal text into Marathi. Ensure all key legal terms like 'possession', 'Seller', 'Buyer', and 'government dues' \n are correctly translated. Maintain full context and structure. If a term has multiple possible translations, choose the one used in Indian legal documents.<br>
0.29
<br><br>

***In this i have observed that if i give more weight to unigram that is for single word then the accuracy of the model increases***

In [5]:
!pip install evaluate sacrebleu bert-score nltk


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 75.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 63.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 26.6 MB/s eta

In [6]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Without Normalizing

In [10]:
from nltk.translate.bleu_score import sentence_bleu
import evaluate  # Hugging Face metrics

# Load metrics
bertscore = evaluate.load("bertscore")
chrf = evaluate.load("chrf")
ter = evaluate.load("ter")

# ---------------- Gemini Translate ----------------
def gemini_translate(text: str) -> str:
    response = gemini_client.models.generate_content(
        model="gemini-2.0-flash",
        contents=f"""
        Translate the following English legal text into Marathi in a formal legal register. 
        Use terminology consistent with Indian contracts and deeds. 
        Do not simplify or modernize the language. 
        Preserve original clause numbering, structure, and formatting. 
        Output only the translated document, nothing else.

        {text}""",
        config=types.GenerateContentConfig(
            temperature=0.0,
            top_p=1.0,
            top_k=1,
            max_output_tokens=2048
        )
    )
    return response.text


# ---------------- Run Translations ----------------
results = []

for prompt in prompts:  
    # --- Gemini Translation ---
    gemini_translation = gemini_translate(f"{prompt}\n\n{english_text}")

    # --- BLEU (sentence-level, 0–1 scale) ---
    ref_tokens = reference_translation.split()
    gemini_bleu = sentence_bleu(
        [ref_tokens],
        gemini_translation.split(),
        weights=(0.25, 0.25, 0.25, 0.25)  # balanced n-grams
    )

    # --- ChrF ---
    gemini_chrf = chrf.compute(
        predictions=[gemini_translation],
        references=[reference_translation]
    )["score"]

    # --- TER ---
    gemini_ter = ter.compute(
        predictions=[gemini_translation],
        references=[reference_translation]
    )["score"]

    # --- BERTScore (F1 only) ---
    gemini_bertscore = bertscore.compute(
        predictions=[gemini_translation],
        references=[reference_translation],
        lang="mr"   # Marathi
    )["f1"][0]

    results.append({
        "English Text": english_text[:100] + "...",
        "Prompt": prompt,
        "Google Gemini": gemini_translation,
        "BLEU": round(gemini_bleu, 2),
        "ChrF": round(gemini_chrf, 2),
        "TER": round(gemini_ter, 2),
        "BERTScore (F1)": round(gemini_bertscore, 2),
        "Reference Translation": reference_translation[:100] + "..."
    })

# ---------------- Show Results ----------------
df = pd.DataFrame(results)
pd.set_option('display.max_colwidth', None)
display(HTML(df.to_html(escape=False)))

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

,English Text,Prompt,Google Gemini,BLEU,ChrF,TER,BERTScore (F1),Reference Translation
0,"\nSALE DEED\nThis Sale Deed is executed on this ___ day of ____, 20 at [Place], Maharashtra.\nBY AND BE...",Translate the following text into Marathi exactly preserving legal terms.,"विक्रयपत्र\nहे विक्रयपत्र आज दिनांक ___ रोजी ____, इ.स. २०__ रोजी [ठिकाण], महाराष्ट्र येथे निष्पादित केले आहे.\nयाद्वारे आणि यांच्यामध्ये:\n१. [विक्रेत्याचे नाव], वय अंदाजे ___ वर्षे, रा. [पत्ता], यापुढे “विक्रेता” म्हणून संदर्भित (ज्यामध्ये, संदर्भास किंवा अर्थास विसंगत नसल्यास, त्यांचे वारस, कायदेशीर प्रतिनिधी, निष्पादक, प्रशासक आणि हक्क हस्तांतरिती यांचा समावेश असेल).\nआणि\n२. [खरेदीदाराचे नाव], वय अंदाजे ___ वर्षे, रा. [पत्ता], यापुढे “खरेदीदार” म्हणून संदर्भित (ज्यामध्ये, संदर्भास किंवा अर्थास विसंगत नसल्यास, त्यांचे वारस, कायदेशीर प्रतिनिधी, निष्पादक, प्रशासक आणि हक्क हस्तांतरिती यांचा समावेश असेल).\nविक्रेता आणि खरेदीदार यांना एकत्रितपणे “पक्षकार” म्हणून संबोधले जाईल.\n________________________________________\nWHEREAS (म्हणजे):\n१. विक्रेता भूखंडाचे संपूर्ण आणि कायदेशीर मालक आहे, ज्या भूखंडाचा सर्व्हे नंबर [____] आहे, आणि त्याचे क्षेत्रफळ [____] चौ.मी./चौ.फूट आहे, जो [गाव/शहर, तालुका, जिल्हा, महाराष्ट्र] येथे स्थित आहे, यापुढे “सदर मालमत्ता” म्हणून संदर्भित केला जाईल.\n२. विक्रेत्याने सदर मालमत्ता खरेदीदाराला एकूण ₹[रक्कम] (रुपये [अक्षरात रक्कम]) च्या मोबदल्यात विक्री करण्याचे मान्य केले आहे.\n३. खरेदीदाराने सदर मालमत्ता विक्रेत्याकडून उपरोक्त मोबदल्यात, कोणत्याही भार, धारणाधिकार, शुल्क, गहाणखत किंवा खटलाबाजी विरहित खरेदी करण्याचे मान्य केले आहे.\n________________________________________\nआता हे দলিল साक्ष देते की:\n१. विक्री आणि हस्तांतरण:\nविक्रेता, उपरोक्त नमूद केलेल्या मोबदल्याच्या बदल्यात, सदर मालमत्ता सर्व हक्क, सुखाधिकार आणि उपांगिकासह (appurtenances) खरेदीदाराला पूर्णपणे विकतो, हस्तांतरित करतो आणि प्रदान करतो.\n२. मोबदला:\nएकूण विक्री मोबदला ₹[रक्कम] विक्रेत्याने खरेदीदाराकडून [भरणा करण्याची पद्धत: धनादेश/रोख/बँक हस्तांतरण] द्वारे प्राप्त केला आहे, ज्याची पावती विक्रेता याद्वारे कबूल करतो.\n३. ताबे:\nविक्रेता याद्वारे सदर मालमत्तेचा प्रत्यक्ष आणि कायदेशीर ताबा या दस्ताच्या निष्पादनावर खरेदीदाराला देतो.\n४. मालकी हक्क आणि हमी:\nविक्रेता खात्री देतो की सदर मालमत्ता सर्व भार, कायदेशीर विवाद, दावे किंवा गहाणखतांपासून मुक्त आहे. विक्रेता पुढे सहमत आहे की सदर मालमत्तेच्या मालकी हक्कामध्ये कोणत्याही दोषांमुळे उद्भवलेल्या कोणत्याही नुकसानी भरपाई तो खरेदीदारास देईल.\n५. खर्च आणि नोंदणी:\n मुद्रांक शुल्क, नोंदणी शुल्क आणि इतर आनुषंगिक खर्चाशी संबंधित सर्व खर्च [खरेदीदार/विक्रेता यांच्यात परस्पर मान्यतेनुसार] द्वारे वहन केले जातील.\n६. शासित कायदा:\nहे विक्रयपत्र भारताच्या कायद्यानुसार आणि विशेषतः महाराष्ट्र मुद्रांक अधिनियम (Maharashtra Stamp Act) आणि मालमत्ता हस्तांतरण अधिनियम, १८८२ (Transfer of Property Act, 1882) नुसार शासित आणि अन्वयार्थ लावले जाईल.\n________________________________________\nयाची साक्ष म्हणून, विक्रेता आणि खरेदीदार यांनी हे विक्रयपत्र वर नमूद केलेल्या दिवशी, महिन्यात आणि वर्षी निष्पादित केले आहे.\nसही, शिक्का व सुपूर्द:\nविक्रेता:\n(सही) ___________________\nनाव: _______________________\nपत्ता: ____________________\nखरेदीदार:\n(सही) ___________________\nनाव: _______________________\nपत्ता: ____________________\n साक्षीदार:\n१. नाव: ___________________ सही: _______________ पत्ता: _______________\n२. नाव: ___________________ सही: _______________ पत्ता: _______________\n",0.23,70.44,61.46,0.85,\nविक्री करार\nहे विक्री करार २० तारखेला महाराष्ट्रातील [स्थान] येथे ____ दिवशी अंमलात आणले जात आहे .\n...
1,"\nSALE DEED\nThis Sale Deed is executed on this ___ day of ____, 20 at [Place], Maharashtra.\nBY AND BE...","Translate the following text into Marathi in simple, easy-to-understand Marathi.","**विक्रयपत्र**\n\nहे विक्रयपत्र आज दिनांक ____, ____, २० रोजी [ठिकाण], महाराष्ट्र येथे निष्पादित केले जात आहे.\n\nयाद्वारे:\n\n1. [विक्रेत्याचे नाव], वय अंदाजे ___ वर्षे, रा. [पत्ता], यापुढे ""विक्रेता"" म्हणून संदर्भित (ज्यात संदर्भानुसार अन्यथा अर्थ अपेक्षित नसल्यास, त्यांचे वारस, कायदेशीर प्रतिनिधी, निष्पादक, प्रशासक आणि हक्क हस्तांतरिती यांचा समावे

**In this code i have translated english text into marathi using gemini 2.0 flash model**<br>
Evaluation metrics used - <br>
1. BLEU - compares n-gram overlap with reference
2. ChrF - compares character-level similarity
3. TER - counts how many edits are needed to match reference
4. BERTScore (F1) - checks meaning similarity using embeddings

> **Google 2.0 flash score -** <br>
>1. Highest BLEU (0.28)
2. Highest ChrF (73.46)
3. Lowest TER (55.26)
4. Good BERTScore (0.86) <br>
> ***Observation - The models BLEU score decreases when normalization is not performed***

# With Normalizing

In [12]:
from nltk.translate.bleu_score import sentence_bleu
import evaluate  # Hugging Face metrics
from sacrebleu.tokenizers import tokenizer_13a  # normalization

# Load metrics
bertscore = evaluate.load("bertscore")
chrf = evaluate.load("chrf")
ter = evaluate.load("ter")

# SacreBLEU tokenizer
tok = tokenizer_13a.Tokenizer13a()


# ---------------- Gemini Translate ----------------
def gemini_translate(text: str) -> str:
    response = gemini_client.models.generate_content(
        model="gemini-2.0-flash",
        contents=f"""
        Translate the following English legal text into Marathi in a formal legal register. 
        Use terminology consistent with Indian contracts and deeds. 
        Do not simplify or modernize the language. 
        Preserve original clause numbering, structure, and formatting. 
        Output only the translated document, nothing else.

        {text}""",
        config=types.GenerateContentConfig(
            temperature=0.0,
            top_p=1.0,
            top_k=1,
            max_output_tokens=2048
        )
    )
    return response.text


# ---------------- Run Translations ----------------
results = []

for prompt in prompts:  
    # --- Gemini Translation ---
    gemini_translation = gemini_translate(f"{prompt}\n\n{english_text}")

    # --- Normalize both system and reference outputs ---
    norm_translation = tok(gemini_translation)
    norm_reference = tok(reference_translation)

    # --- BLEU (sentence-level, 0–1 scale) ---
    ref_tokens = norm_reference.split()
    gemini_bleu = sentence_bleu(
        [ref_tokens],
        norm_translation.split(),
        weights=(0.25, 0.25, 0.25, 0.25)
    )

    # --- ChrF ---
    gemini_chrf = chrf.compute(
        predictions=[norm_translation],
        references=[norm_reference]
    )["score"]

    # --- TER ---
    gemini_ter = ter.compute(
        predictions=[norm_translation],
        references=[norm_reference]
    )["score"]

    # --- BERTScore (F1 only, use raw text not normalized) ---
    gemini_bertscore = bertscore.compute(
        predictions=[gemini_translation],
        references=[reference_translation],
        lang="mr"
    )["f1"][0]

    results.append({
        "English Text": english_text[:100] + "...",
        "Prompt": prompt,
        "Google Gemini": gemini_translation,
        "BLEU": round(gemini_bleu, 2),
        "ChrF": round(gemini_chrf, 2),
        "TER": round(gemini_ter, 2),
        "BERTScore (F1)": round(gemini_bertscore, 2),
        "Reference Translation": reference_translation[:100] + "..."
    })

# ---------------- Show Results ----------------
df = pd.DataFrame(results)
pd.set_option('display.max_colwidth', None)
display(HTML(df.to_html(escape=False)))

,English Text,Prompt,Google Gemini,BLEU,ChrF,TER,BERTScore (F1),Reference Translation
0,"\nSALE DEED\nThis Sale Deed is executed on this ___ day of ____, 20 at [Place], Maharashtra.\nBY AND BE...",Translate the following text into Marathi exactly preserving legal terms.,"विक्रयपत्र\nहे विक्रयपत्र आज दिनांक ___ रोजी ____, इ.स. २०__ रोजी [ठिकाण], महाराष्ट्र येथे निष्पादित केले आहे.\nयाद्वारे आणि यांच्यामध्ये:\n१. [विक्रेत्याचे नाव], वय अंदाजे ___ वर्षे, रा. [पत्ता], यापुढे “विक्रेता” म्हणून संदर्भित (ज्यामध्ये, संदर्भास किंवा अर्थास विसंगत नसल्यास, त्यांचे वारस, कायदेशीर प्रतिनिधी, निष्पादक, प्रशासक आणि हक्क हस्तांतरिती यांचा समावेश असेल).\nआणि\n२. [खरेदीदाराचे नाव], वय अंदाजे ___ वर्षे, रा. [पत्ता], यापुढे “खरेदीदार” म्हणून संदर्भित (ज्यामध्ये, संदर्भास किंवा अर्थास विसंगत नसल्यास, त्यांचे वारस, कायदेशीर प्रतिनिधी, निष्पादक, प्रशासक आणि हक्क हस्तांतरिती यांचा समावेश असेल).\nविक्रेता आणि खरेदीदार यांना एकत्रितपणे “पक्षकार” म्हणून संबोधले जाईल.\n________________________________________\nWHEREAS (म्हणजे):\n१. विक्रेता भूखंडाचे संपूर्ण आणि कायदेशीर मालक आहे, ज्या भूखंडाचा सर्व्हे नंबर [____] आहे, आणि त्याचे क्षेत्रफळ [____] चौ.मी./चौ.फूट आहे, जो [गाव/शहर, तालुका, जिल्हा, महाराष्ट्र] येथे स्थित आहे, यापुढे “सदर मालमत्ता” म्हणून संदर्भित केला जाईल.\n२. विक्रेत्याने सदर मालमत्ता खरेदीदाराला एकूण ₹[रक्कम] (रुपये [अक्षरात रक्कम]) च्या मोबदल्यात विक्री करण्याचे मान्य केले आहे.\n३. खरेदीदाराने सदर मालमत्ता विक्रेत्याकडून उपरोक्त मोबदल्यात, कोणत्याही भार, धारणाधिकार, शुल्क, गहाणखत किंवा खटलाबाजी विरहित खरेदी करण्याचे मान्य केले आहे.\n________________________________________\nआता हे দলিল साक्ष देते की:\n१. विक्री आणि हस्तांतरण:\nविक्रेता, उपरोक्त नमूद केलेल्या मोबदल्याच्या बदल्यात, सदर मालमत्ता सर्व हक्क, सुखाधिकार आणि उपांगिकासह (appurtenances) खरेदीदाराला पूर्णपणे विकतो, हस्तांतरित करतो आणि प्रदान करतो.\n२. मोबदला:\nएकूण विक्री मोबदला ₹[रक्कम] विक्रेत्याने खरेदीदाराकडून [भरणा करण्याची पद्धत: धनादेश/रोख/बँक हस्तांतरण] द्वारे प्राप्त केला आहे, ज्याची पावती विक्रेता याद्वारे कबूल करतो.\n३. ताबे:\nविक्रेता याद्वारे सदर मालमत्तेचा प्रत्यक्ष आणि कायदेशीर ताबा या दस्ताच्या निष्पादनावर खरेदीदाराला देतो.\n४. मालकी हक्क आणि हमी:\nविक्रेता खात्री देतो की सदर मालमत्ता सर्व भार, कायदेशीर विवाद, दावे किंवा गहाणखतांपासून मुक्त आहे. विक्रेता पुढे सहमत आहे की सदर मालमत्तेच्या मालकी हक्कामध्ये कोणत्याही दोषांमुळे उद्भवलेल्या कोणत्याही नुकसानी भरपाई तो खरेदीदारास देईल.\n५. खर्च आणि नोंदणी:\n मुद्रांक शुल्क, नोंदणी शुल्क आणि इतर आनुषंगिक खर्चाशी संबंधित सर्व खर्च [खरेदीदार/विक्रेता यांच्यात परस्पर मान्यतेनुसार] द्वारे वहन केले जातील.\n६. शासित कायदा:\nहे विक्रयपत्र भारताच्या कायद्यानुसार आणि विशेषतः महाराष्ट्र मुद्रांक अधिनियम (Maharashtra Stamp Act) आणि मालमत्ता हस्तांतरण अधिनियम, १८८२ (Transfer of Property Act, 1882) नुसार शासित आणि अन्वयार्थ लावले जाईल.\n________________________________________\nयाची साक्ष म्हणून, विक्रेता आणि खरेदीदार यांनी हे विक्रयपत्र वर नमूद केलेल्या दिवशी, महिन्यात आणि वर्षी निष्पादित केले आहे.\nसही, शिक्का व सुपूर्द:\nविक्रेता:\n(सही) ___________________\nनाव: _______________________\nपत्ता: ____________________\nखरेदीदार:\n(सही) ___________________\nनाव: _______________________\nपत्ता: ____________________\n साक्षीदार:\n१. नाव: ___________________ सही: _______________ पत्ता: _______________\n२. नाव: ___________________ सही: _______________ पत्ता: _______________\n",0.63,70.44,30.72,0.85,\nविक्री करार\nहे विक्री करार २० तारखेला महाराष्ट्रातील [स्थान] येथे ____ दिवशी अंमलात आणले जात आहे .\n...
1,"\nSALE DEED\nThis Sale Deed is executed on this ___ day of ____, 20 at [Place], Maharashtra.\nBY AND BE...","Translate the following text into Marathi in simple, easy-to-understand Marathi.","**विक्रयपत्र**\n\nहे विक्रयपत्र आज दिनांक ____, ____, २० रोजी [ठिकाण], महाराष्ट्र येथे निष्पादित केले आहे.\n\nयाद्वारे:\n\n1. [विक्रेत्याचे नाव], वय अंदाजे ___ वर्षे, रा. [पत्ता], यापुढे ""विक्रेता"" म्हणून संदर्भित (ज्यात संदर्भानुसार अन्यथा अर्थ नसेल तर त्यांचे वारस, कायदेशीर प्रतिनिधी, निष्पादक, प्रशासक आणि नियुक्त यांचा समावेश असेल).\n\nआणि\n\n2. [

**In this code i have translated english text into marathi using gemini 2.0 flash model with normalizing generated text**<br>
Evaluation metrics used - <br>
1. BLEU - compares n-gram overlap with reference
2. ChrF - compares character-level similarity
3. TER - counts how many edits are needed to match reference
4. BERTScore (F1) - checks meaning similarity using embeddings

> **Google Gemini 2.0 flash -** <br>
1. Highest BLEU (0.68) 
2. high ChrF (71.75)
3. low TER (29.54)
4. Best BERTScore (0.87)

In [18]:

from nltk.translate.bleu_score import sentence_bleu
import evaluate  # Hugging Face metrics
from sacrebleu.tokenizers import tokenizer_13a  # normalization

# Load metrics
bertscore = evaluate.load("bertscore")
chrf = evaluate.load("chrf")
ter = evaluate.load("ter")

# SacreBLEU tokenizer
tok = tokenizer_13a.Tokenizer13a()


# ---------------- Gemini Translate ----------------
def gemini_translate(text: str) -> str:
    response = gemini_client.models.generate_content(
        model="gemini-2.5-flash-lite",
        contents=f"""
        Translate the following English legal text into Marathi in a formal legal register. 
        Use terminology consistent with Indian contracts and deeds. 
        Do not simplify or modernize the language. 
        Preserve original clause numbering, structure, and formatting. 
        Output only the translated document, nothing else.

        {text}""",
        config=types.GenerateContentConfig(
            temperature=0.0,
            top_p=1.0,
            top_k=1,
            max_output_tokens=2048
        )
    )
    # ✅ Make sure we return a clean string, never None
    return response.text if response and response.text else ""



# ---------------- Run Translations ----------------
results = []

for prompt in prompts:  
    # --- Gemini Translation ---
    gemini_translation = gemini_translate(f"{prompt}\n\n{english_text}")

    # --- Normalize both system and reference outputs ---
    norm_translation = tok(gemini_translation)
    norm_reference = tok(reference_translation)

    # --- BLEU (sentence-level, 0–1 scale) ---
    ref_tokens = norm_reference.split()
    gemini_bleu = sentence_bleu(
        [ref_tokens],
        norm_translation.split(),
        weights=(0.25, 0.25, 0.25, 0.25) # assigned equal weights
    )

    # --- ChrF ---
    gemini_chrf = chrf.compute(
        predictions=[norm_translation],
        references=[norm_reference]
    )["score"]

    # --- TER ---
    gemini_ter = ter.compute(
        predictions=[norm_translation],
        references=[norm_reference]
    )["score"]

    # --- BERTScore (F1 only, use raw text not normalized) ---
    gemini_bertscore = bertscore.compute(
        predictions=[gemini_translation],
        references=[reference_translation],
        lang="mr"
    )["f1"][0]

    results.append({
        "English Text": english_text[:100] + "...",
        "Prompt": prompt,
        "Google Gemini Pro": gemini_translation,
        "BLEU": round(gemini_bleu, 2),
        "ChrF": round(gemini_chrf, 2),
        "TER": round(gemini_ter, 2),
        "BERTScore (F1)": round(gemini_bertscore, 2),
        "Reference Translation": reference_translation[:100] + "..."
    })

# ---------------- Show Results ----------------
df = pd.DataFrame(results)
pd.set_option('display.max_colwidth', None)
display(HTML(df.to_html(escape=False)))

,English Text,Prompt,Google Gemini Pro,BLEU,ChrF,TER,BERTScore (F1),Reference Translation
0,"\nSALE DEED\nThis Sale Deed is executed on this ___ day of ____, 20 at [Place], Maharashtra.\nBY AND BE...",Translate the following text into Marathi exactly preserving legal terms.,"विक्री करारनामा\nहा विक्री करारनामा आज दिनांक ____ ____, २०२१ रोजी [ठिकाण], महाराष्ट्र येथे निष्पादित करण्यात आला आहे.\n\nआणि यांच्यामध्ये\n\n१. [विक्रेत्याचे नाव], वय अंदाजे ___ वर्षे, पत्ता [पत्ता], यापुढे ""विक्रेता"" म्हणून संबोधले जाईल (ज्या अभिव्यक्तीमध्ये, संदर्भ किंवा अर्थाला प्रतिकूल नसल्यास, त्याचे/तिचे वारस, कायदेशीर प्रतिनिधी, विश्वस्त, प्रशासक आणि नियुक्त व्यक्ती यांचा समावेश असेल).\n\nआणि\n\n२. [खरेदीदाराचे नाव], वय अंदाजे ___ वर्षे, पत्ता [पत्ता], यापुढे ""खरेदीदार"" म्हणून संबोधले जाईल (ज्या अभिव्यक्तीमध्ये, संदर्भ किंवा अर्थाला प्रतिकूल नसल्यास, त्याचे/तिचे वारस, कायदेशीर प्रतिनिधी, विश्वस्त, प्रशासक आणि नियुक्त व्यक्ती यांचा समावेश असेल).\n\nविक्रेता आणि खरेदीदार यांना एकत्रितपणे ""पक्षकार"" म्हणून संबोधले जाईल.\n________________________________________\nज्याअर्थी:\n१. विक्रेता हा सर्वेक्षण क्रमांक [____] चा, [गाव/शहर, तालुका, जिल्हा, महाराष्ट्र] येथे स्थित, अंदाजे [____] चौ.मी./चौ.फूट क्षेत्रफळाचा भूखंड, यापुढे ""सदर मालमत्ता"" म्हणून संबोधला जाईल, याचा पूर्ण आणि कायदेशीर मालक आहे.\n२. विक्रेत्याने सदर मालमत्ता खरेदीदारास एकूण ₹[रक्कम] (रुपये [शब्दात रक्कम]) या मोबदल्यात विकण्यास सहमती दर्शविली आहे.\n३. खरेदीदाराने सदर मालमत्ता विक्रेत्याकडून नमूद केलेल्या मोबदल्यात, सर्व बोजा, धारणाधिकार, शुल्क, गहाण किंवा खटल्यांपासून मुक्त खरेदी करण्यास सहमती दर्शविली आहे.\n________________________________________\nआता हा करारनामा खालीलप्रमाणे साक्ष देतो:\n\n१. विक्री आणि हस्तांतरण:\nविक्रेत्याने, वर नमूद केलेल्या मोबदल्यासाठी, सदर मालमत्ता, त्यासोबतचे सर्व अधिकार, सुखसोयी आणि संलग्न गोष्टींसह, खरेदीदारास पूर्णपणे विकल्या आहेत, हस्तांतरित केल्या आहेत आणि सुपूर्द केल्या आहेत.\n\n२. मोबदला:\n₹[रक्कम] चा एकूण विक्री मोबदला विक्रेत्याला खरेदीदाराकडून [देयकाचा प्रकार: चेक/रोख/बँक हस्तांतरण] द्वारे प्राप्त झाला आहे, ज्याची पोचपावती विक्रेता याद्वारे देतो.\n\n३. ताबा:\nविक्रेत्याने या कराराच्या निष्पादनावर सदर मालमत्तेचा प्रत्यक्ष आणि कायदेशीर ताबा खरेदीदारास याद्वारे सुपूर्द केला आहे.\n\n४. मालकी हक्क आणि हमी:\nविक्रेता खात्री देतो की सदर मालमत्ता सर्व बोजा, कायदेशीर वाद, दावे किंवा गहाणांपासून मुक्त आहे. सदर मालमत्तेच्या मालकी हक्कात कोणताही दोष आढळल्यास त्यामुळे होणाऱ्या कोणत्याही नुकसानीपासून खरेदीदाराला नुकसानभरपाई देण्यास विक्रेता पुढे सहमत आहे.\n\n५. खर्च आणि नोंदणी:\nमुद्रांक शुल्क, नोंदणी शुल्क आणि इतर संबंधित खर्चांशी संबंधित सर्व खर्च [खरेदीदार/विक्रेता परस्पर सहमत] द्वारे उचलले जातील.\n\n६. लागू कायदा:\nहा विक्री करारनामा भारताच्या कायद्यांनुसार, आणि विशेषतः महाराष्ट्र मुद्रांक अधिनियम आणि मालमत्ता हस्तांतरण अधिनियम, १८८२ नुसार शासित आणि अर्थ लावला जाईल.\n________________________________________\nयाची साक्ष म्हणून, विक्रेत्याने आणि खरेदीदाराने वर नमूद केलेल्या दिवस, महिना आणि वर्षी हा विक्री करारनामा निष्पादित केला आहे.\n\nयाद्वारे निष्पादित, मुद्रांकित आणि सुपूर्द केलेले:\n\nविक्रेता:\n(सही) ___________________\nनाव: _______________________\nपत्ता: ____________________\n\nखरेदीदार:\n(सही) ___________________\nनाव: _______________________\nपत्ता: ____________________\n\nसाक्षीदार:\n१. नाव: ___________________ सही: _______________ पत्ता: _______________\n२. नाव: ___________________ सही: _______________ पत्ता: _______________",0.65,72.47,26.26,0.88,\nविक्री करार\nहे विक्री करार २० तारखेला महाराष्ट्रातील [स्थान] येथे ____ दिवशी अंमलात आणले जात आहे .\n...
1,"\nSALE DEED\nThis Sale Deed is executed on this ___ day of ____, 20 at [Place], Maharashtra.\nBY AND BE...","Translate the following text into Marathi in simple, easy-to-understand Marathi.","विक्री करारनामा\nहा विक्री करारनामा आज दिनांक ____ ____, २०२१ रोजी [ठिकाण], महाराष्ट्र येथे निष्पादित करण्यात आला आहे.\n\n**यांच्यामध्ये**\n\n१. [विक्रेत्याचे नाव], वय अंदाजे ___ वर्षे, पत्ता [पत्ता], यापुढे ""विक्रेता"" म्हणून संबोधले जाईल (ज्याचा अर्थ, संदर्भ किंवा अर्थाला प्रतिकूल नसल्यास, त्याचे/तिचे वार

**In this code i have translated english text into marathi using gemini 2.0 flash lite model with normalizing generated text**<br>
Evaluation metrics used - <br>
1. BLEU - compares n-gram overlap with reference
2. ChrF - compares character-level similarity
3. TER - counts how many edits are needed to match reference
4. BERTScore (F1) - checks meaning similarity using embeddings

> **Google Gemini flash lite score -** <br>
**Prompt -** Translate the following English legal text into Marathi exactly preserving all legal terms and clause structures. \n Use precise legal terminology. Do not simplify or reword any clauses. Maintain numbering, punctuation, and formatting as in the original text.<br>
>1. High BLEU (0.68)
2. Strong ChrF (74.14)
3. Lowest TER (23.92)
4. High BERTScore (0.88)

> *This score is best because it has high BLEU score 68%, chrf (character n-gram f score) counts overlaps of characters higher chrf is better score..TER (Translation edit rate) should be minimum and in this 23 is good score because it is less than < 30.. high bert score 0.88 (for scale 0 to 1) is better it checcks semantic meaning of generated translation with reference translation instead of matching exact words.*


>>